# 1. How to import PySpark and check the version?

In [0]:
#import findspark
#findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Pyspark Concepts").getOrCreate()
print(spark.version)


3.1.2

# 2. How to convert the index of a PySpark DataFrame into a column?
- Hint: The PySpark DataFrame doesn’t have an explicit concept of an index like Pandas DataFrame. However, if you have a DataFrame and you’d like to add a new column that is basically a row number.

In [0]:
df = spark.createDataFrame([
("Alice", 1),
("Bob", 2),
("Charlie", 3),
], ["Name", "Value"])

df.display()


Name,Value
Alice,1
Bob,2
Charlie,3


In [0]:
from pyspark.sql.functions import row_number,lit,monotonically_increasing_id
from pyspark.sql import Window
w= Window().orderBy(lit('Name'))
#or
w= Window().orderBy(monotonically_increasing_id())
df_index = df.withColumn('Index',row_number().over(w) -1)

df_index.display()

Name,Value,Index
Alice,1,0
Bob,2,1
Charlie,3,2


# 3. How to combine many lists to form a PySpark DataFrame?
- Create a PySpark DataFrame from list1 and list2

- Hint: For Creating DataFrame from multiple lists, first create an RDD (Resilient Distributed Dataset) from those lists and then convert the RDD to a DataFrame.

In [0]:
#combine many lists
# Define your lists
list1 = ["a", "b", "c", "d"]
list2 = [1, 2, 3, 4]

In [0]:

rdd_ser = spark.sparkContext.parallelize(list(zip(list1,list2)))
df = rdd_ser.toDF(["column1","column2"])
df.display()


column1,column2
a,1
b,2
c,3
d,4


In [0]:
#sort a list
list1.sort(reverse=True)
print(list1)

['d', 'c', 'b', 'a']

# 4. How to get the items of list A not present in list B?
- Get the items of list_A not present in list_B in PySpark, you can use the subtract operation on RDDs (Resilient Distributed Datasets).

In [0]:
list_A = [1, 2, 3, 4, 5]
list_B = [4, 5, 6, 7, 8]

In [0]:
#get the items of list A not present in list B
list_A = [1, 2, 3, 4, 5]
list_B = [4, 5, 6, 7, 8]
set_C = set(list_A) - set(list_B)
list_C = [*set_C]
print(list_C)

rdd_A = sc.parallelize(list_A)
rdd_B = sc.parallelize(list_B)
result = rdd_A.subtract(rdd_B).collect()
print(result)

[1, 2, 3]
[1, 2, 3]

#5. How to get the items not common to both list A and list B?
- Get all items of list_A and list_B not common to both.

In [0]:
list_A = [1, 2, 3, 4, 5]
list_B = [4, 5, 6, 7, 8]


In [0]:
###items not common to both list
list_C = list(set(list_A) ^ set(list_B))
print(list_C)

[1, 2, 3, 6, 7, 8]

#6. How to get the minimum, 25th percentile, median, 75th, and max of a numeric column?
- Compute the minimum, 25th percentile, median, 75th, and maximum of column Age

In [0]:
# Create a sample DataFrame
data = [("A", 10), ("B", 20), ("C", 30), ("D", 40), ("E", 50), ("F", 15), ("G", 28), ("H", 54), ("I", 41), ("J", 86)]
df = spark.createDataFrame(data, ["Name", "Age"])

df.show()

+----+---+
Name|Age|
+----+---+
 A| 10|
 B| 20|
 C| 30|
 D| 40|
 E| 50|
 F| 15|
 G| 28|
 H| 54|
 I| 41|
 J| 86|
+----+---+

In [0]:
###minimum, 25th percentile, median, 75th, and max
from pyspark.sql.functions import min,max,avg,percentile_approx
df_min = df.select(min('Age').alias('Minimum'),
                   percentile_approx('Age',0.25).alias('Percentile_25'),
                   avg('Age').alias('Average'),
                   percentile_approx('Age',0.50).alias('Median'),
                   percentile_approx('Age',0.75).alias('Percentile_75'),
                   max('Age').alias('Maximum'))

df_min.display()

Minimum,Percentile_25,Average,Median,Percentile_75,Maximum
10,20,37.4,30,50,86


#7. How to get frequency counts of unique items of a column?
- Calculte the frequency counts of each unique value

In [0]:
from pyspark.sql import Row

# Sample data
data = [
Row(name='John', job='Engineer'),
Row(name='John', job='Engineer'),
Row(name='Mary', job='Scientist'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Scientist'),
Row(name='Sam', job='Doctor'),
]

# create DataFrame
df = spark.createDataFrame(data)

# show DataFrame
df.show()

+----+---------+
name| job|
+----+---------+
John| Engineer|
John| Engineer|
Mary|Scientist|
 Bob| Engineer|
 Bob| Engineer|
 Bob|Scientist|
 Sam| Doctor|
+----+---------+

In [0]:
###frequency counts
df_freq = df.groupBy('job').count()

df_freq.display()

job,count
Engineer,4
Scientist,2
Doctor,1


#8. How to keep only top 2 most frequent values as it is and replace everything else as ‘Other’?

In [0]:
from pyspark.sql import Row

# Sample data
data = [
Row(name='John', job='Engineer'),
Row(name='John', job='Engineer'),
Row(name='Mary', job='Scientist'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Scientist'),
Row(name='Sam', job='Doctor'),
]

# create DataFrame
df = spark.createDataFrame(data)

# show DataFrame
df.show()

+----+---------+
name| job|
+----+---------+
John| Engineer|
John| Engineer|
Mary|Scientist|
 Bob| Engineer|
 Bob| Engineer|
 Bob|Scientist|
 Sam| Doctor|
+----+---------+

In [0]:
###2 most frequent values as it is and replace everything else as ‘Other’
from pyspark.sql.functions import desc,col,when,lit

df_tmp = df.groupBy('job').count().alias('count')
df_tmp = df_tmp.orderBy(col('count').desc()).limit(2)
df_tmp = df.join(df_tmp,on='job',how='left')
df_tmp = df_tmp.withColumn('new',when(col('count')> 0,col('job')).otherwise(lit('Other'))).drop('count')


df_tmp.display()

job,name,new
Engineer,John,Engineer
Engineer,John,Engineer
Scientist,Mary,Scientist
Engineer,Bob,Engineer
Engineer,Bob,Engineer
Scientist,Bob,Scientist
Doctor,Sam,Other


In [0]:
from pyspark.sql.functions import col, when

# Get the top 2 most frequent jobs
top_2_jobs = df.groupBy('job').count().orderBy('count', ascending=False).limit(2).select('job').rdd.flatMap(lambda x: x).collect()

# Replace all but the top 2 most frequent jobs with 'Other'
df = df.withColumn('job', when(col('job').isin(top_2_jobs), col('job')).otherwise('Other'))

# show DataFrame
df.show()

+----+---------+
name| job|
+----+---------+
John| Engineer|
John| Engineer|
Mary|Scientist|
 Bob| Engineer|
 Bob| Engineer|
 Bob|Scientist|
 Sam| Other|
+----+---------+

#9. How to Drop rows with NA values specific to a particular column?

In [0]:
# Assuming df is your DataFrame
df = spark.createDataFrame([
("A", 1, None),
("B", None, "123" ),
("B", 3, "456"),
("D", None, None),
], ["Name", "Value", "id"])

df.show()

+----+-----+----+
Name|Value| id|
+----+-----+----+
 A| 1|null|
 B| null| 123|
 B| 3| 456|
 D| null|null|
+----+-----+----+

In [0]:
###Drop rows with NA values
df_result = df.filter(col('Value').isNotNull())
#OR
df_result = df.dropna(subset=['Value'])
df_result.display()


Name,Value,id
A,1,null
B,3,456


#10. How to rename columns of a PySpark DataFrame using two lists – one containing the old column names and the other containing the new column names?

In [0]:
# suppose you have the following DataFrame
df = spark.createDataFrame([(1, 2, 3), (4, 5, 6)], ["col1", "col2", "col3"])

# old column names
old_names = ["col1", "col2", "col3"]

# new column names
new_names = ["new_col1", "new_col2", "new_col3"]

df.show()

+----+----+----+
col1|col2|col3|
+----+----+----+
 1| 2| 3|
 4| 5| 6|
+----+----+----+

In [0]:

###rename columns of a PySpark DataFrame using two lists
df_renamed = df.select(*[col(old).alias(new) for old,new in zip(old_names,new_names)])
df_renamed.display()

new_col1,new_col2,new_col3
1,2,3
4,5,6


#11. How to bin a numeric list to 10 groups of equal size?

In [0]:
from pyspark.sql.functions import rand
from pyspark.ml.feature import Bucketizer

# Create a DataFrame with a single column "values" filled with random numbers
num_items = 100
df = spark.range(num_items).select(rand(seed=42).alias("values"))

df.show(5)

+-------------------+
 values|
+-------------------+
 0.619189370225301|
 0.5096018842446481|
 0.8325259388871524|
0.26322809041172357|
 0.6702867696264135|
+-------------------+
only showing top 5 rows

In [0]:
###numeric list to 10 groups of equal size
from pyspark.ml.feature import Bucketizer

num_buckets = 10
quantiles = df.stat.approxQuantile("values", [i/num_buckets for i in range(num_buckets+1)], 0.00)

#splits = [-float('inf'), 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, float('inf')]

bucketizer = Bucketizer(splits=quantiles, inputCol="values", outputCol="binned_values")

df_binned = bucketizer.transform(df)
df_result = df_binned.groupBy('binned_values').count()

df_result.display()


binned_values,count
8.0,10
0.0,9
7.0,10
4.0,10
3.0,10
2.0,10
6.0,10
5.0,10
9.0,11
1.0,10


#12. How to create contigency table?

In [0]:
# Example DataFrame
data = [("A", "X"), ("A", "Y"), ("A", "X"), ("B", "Y"), ("B", "X"), ("C", "X"), ("C", "X"), ("C", "Y")]
df = spark.createDataFrame(data, ["category1", "category2"])

df.show()

+---------+---------+
category1|category2|
+---------+---------+
 A| X|
 A| Y|
 A| X|
 B| Y|
 B| X|
 C| X|
 C| X|
 C| Y|
+---------+---------+

In [0]:
### contigency table
#Frequency:
df_result = df.cube('category1').count()
df_result.display()
#Contigency:
df_result = df.crosstab('category1','category2')
df_result.display()

category1,count
null,8
A,3
B,2
C,3


category1_category2,X,Y
A,2,1
C,2,1
B,1,1


#13. How to find the numbers that are multiples of 3 from a column?

In [0]:
from pyspark.sql.functions import rand

# Generate a DataFrame with a single column "id" with 10 rows
df = spark.range(10)

# Generate a random float between 0 and 1, scale and shift it to get a random integer between 1 and 10
df = df.withColumn("random", ((rand(seed=42) * 10) + 1).cast("int"))

# Show the DataFrame
df.show()

+---+------+
 id|random|
+---+------+
 0| 7|
 1| 9|
 2| 8|
 3| 8|
 4| 3|
 5| 1|
 6| 7|
 7| 4|
 8| 5|
 9| 1|
+---+------+

In [0]:
### find the numbers that are multiples of 3
from pyspark.sql.functions import when, col

df_result = df.withColumn('mult_3', when((col('random') % 3 == 0), 'yes').otherwise('no'))
df_result.display()

id,random,mult_3
0,7,no
1,9,yes
2,8,no
3,8,no
4,3,yes
5,1,no
6,7,no
7,4,no
8,5,no
9,1,no


#14. How to extract items at given positions from a column?

In [0]:
from pyspark.sql.functions import rand

# Generate a DataFrame with a single column "id" with 10 rows
df = spark.range(10)

# Generate a random float between 0 and 1, scale and shift it to get a random integer between 1 and 10
df = df.withColumn("random", ((rand(seed=42) * 10) + 1).cast("int"))

# Show the DataFrame
df.show()

pos = [0, 4, 8, 5]

+---+------+
 id|random|
+---+------+
 0| 7|
 1| 9|
 2| 8|
 3| 8|
 4| 3|
 5| 1|
 6| 7|
 7| 4|
 8| 5|
 9| 1|
+---+------+

In [0]:
### extract items at given positions from a column
pos = [0, 4, 8, 5]
from pyspark.sql.functions import row_number,monotonically_increasing_id
from pyspark.sql import Window
w = Window.orderBy(monotonically_increasing_id())
df_tmp = df.withColumn('Row',row_number().over(w) -1)
df_result = df_tmp.filter(col('Row').isin(pos))
df_result.display()


id,random,Row
0,7,0
4,3,4
5,1,5
8,5,8


#15. How to stack two DataFrames vertically ?

In [0]:
# Create DataFrame for region A
df_A = spark.createDataFrame([("apple", 3, 5), ("banana", 1, 10), ("orange", 2, 8)], ["Name", "Col_1", "Col_2"])
df_A.show()

# Create DataFrame for region B
df_B = spark.createDataFrame([("apple", 3, 5), ("banana", 1, 15), ("grape", 4, 6)], ["Name", "Col_1", "Col_3"])
df_B.show()

+------+-----+-----+
 Name|Col_1|Col_2|
+------+-----+-----+
 apple| 3| 5|
banana| 1| 10|
orange| 2| 8|
+------+-----+-----+

+------+-----+-----+
 Name|Col_1|Col_3|
+------+-----+-----+
 apple| 3| 5|
banana| 1| 15|
 grape| 4| 6|
+------+-----+-----+

In [0]:
### stack two DataFrames vertically
df_result = df_A.unionByName(df_B,allowMissingColumns=True)
df_result.display()

df_result = df_A.union(df_B)
df_result.display()


Name,Col_1,Col_2,Col_3
apple,3,5,null
banana,1,10,null
orange,2,8,null
apple,3,null,5
banana,1,null,15
grape,4,null,6


Name,Col_1,Col_2
apple,3,5
banana,1,10
orange,2,8
apple,3,5
banana,1,15
grape,4,6


#16. How to compute the mean squared error on a truth and predicted columns?

In [0]:
# Assume you have a DataFrame df with two columns "actual" and "predicted"
# For the sake of example, we'll create a sample DataFrame
data = [(1, 1), (2, 4), (3, 9), (4, 16), (5, 25)]
df = spark.createDataFrame(data, ["actual", "predicted"])

df.show()

+------+---------+
actual|predicted|
+------+---------+
 1| 1|
 2| 4|
 3| 9|
 4| 16|
 5| 25|
+------+---------+

In [0]:
### compute the mean squared error on a truth and predicted columns
from pyspark.sql.functions import col, pow, avg

df_tmp = df.withColumn('square',pow(col('actual') -col('predicted'),2))
df_result = df_tmp.select(avg(col('square')).alias('mean_square_err'))
df_result.display()

mean_square_err
116.8


#17. How to convert the first character of each element in a series to uppercase?

In [0]:
# Suppose you have the following DataFrame
data = [("john",), ("alice",), ("bob",)]
df = spark.createDataFrame(data, ["name"])

df.show()


+-----+
 name|
+-----+
 john|
alice|
 bob|
+-----+

In [0]:
### first character of each element in a series to uppercase
from pyspark.sql.functions import initcap, col,udf
from pyspark.sql.types import StringType

df_result = df.withColumn("Name_new",initcap(col("name")))
#df_result.display()

def cap_string(_string):
    return " ".join(word.capitalize() for word in _string.split())
cap_string_udf=udf(cap_string,StringType())

df_result=df.withColumn('new',cap_string_udf('name'))
df_result.display()

name,new
john,John
alice,Alice
bob,Bob


#18. How to compute summary statistics for all columns in a dataframe

In [0]:
# For the sake of example, we'll create a sample DataFrame
data = [('James', 34, 55000),
('Michael', 30, 70000),
('Robert', 37, 60000),
('Maria', 29, 80000),
('Jen', 32, 65000)]

df = spark.createDataFrame(data, ["name", "age" , "salary"])

df.show()

+-------+---+------+
 name|age|salary|
+-------+---+------+
 James| 34| 55000|
Michael| 30| 70000|
 Robert| 37| 60000|
 Maria| 29| 80000|
 Jen| 32| 65000|
+-------+---+------+

In [0]:
###compute summary statistics for all columns
from pyspark.sql.functions import *
df_result = df.summary()
df_result = df.summary('mean','max')
df_result.display()

summary,name,age,salary
mean,null,32.4,66000.0
max,Robert,37,80000


#19. How to calculate the number of characters in each word in a column?

In [0]:
# Suppose you have the following DataFrame
data = [("john",), ("alice",), ("bob",)]
df = spark.createDataFrame(data, ["name"])

df.show()

+-----+
 name|
+-----+
 john|
alice|
 bob|
+-----+

In [0]:
###calculate the number of characters
#from pyspark.sql.functions import char_length,length
df_result = df.withColumn("n_char",length("name"))
df_result.display()

name,n_char
john,4
alice,5
bob,3


#20 How to compute difference of differences between consecutive numbers of a column?

In [0]:
# For the sake of example, we'll create a sample DataFrame
data = [('James', 34, 55000),
('Michael', 30, 70000),
('Robert', 37, 60000),
('Maria', 29, 80000),
('Jen', 32, 65000)]

df = spark.createDataFrame(data, ["name", "age" , "salary"])

df.show()

+-------+---+------+
 name|age|salary|
+-------+---+------+
 James| 34| 55000|
Michael| 30| 70000|
 Robert| 37| 60000|
 Maria| 29| 80000|
 Jen| 32| 65000|
+-------+---+------+

In [0]:
### compute difference of differences between consecutive numbers of a column
from pyspark.sql.functions import monotonically_increasing_id, lag,col,when
from pyspark.sql import Window

df_tmp = df.withColumn("id",monotonically_increasing_id())
w = Window.orderBy("id")
df_tmp = df_tmp.withColumn("prev_salary", lag("salary").over(w))
df_tmp = df_tmp.withColumn("tmp_diff",col("salary") - col("prev_salary"))
df_tmp = df_tmp.withColumn("diff",when(col("tmp_diff").isNull(),0).otherwise(col("tmp_diff")))
df_result = df_tmp.drop("tmp_diff")
df_result.display()

name,age,salary,id,prev_salary,diff
James,34,55000,8589934592,null,0
Michael,30,70000,25769803776,55000,15000
Robert,37,60000,34359738368,70000,-10000
Maria,29,80000,51539607552,60000,20000
Jen,32,65000,60129542144,80000,-15000


#21. How to get the day of month, week number, day of year and day of week from a date strings?

In [0]:
# example data
data = [("2023-05-18","01 Jan 2010",), ("2023-12-31", "01 Jan 2010",)]
df = spark.createDataFrame(data, ["date_str_1", "date_str_2"])

df.show()

+----------+-----------+
date_str_1| date_str_2|
+----------+-----------+
2023-05-18|01 Jan 2010|
2023-12-31|01 Jan 2010|
+----------+-----------+

In [0]:
### day of month, week number, day of year and day of week from a date strings
from pyspark.sql.functions import to_date, dayofmonth, weekofyear, dayofyear, dayofweek

# Convert date string to date format
df = df.withColumn("date_1", to_date(df.date_str_1, 'yyyy-MM-dd'))
df = df.withColumn("date_2", to_date(df.date_str_2, 'dd MMM yyyy'))

df_result = df.withColumn("day_of_month", dayofmonth(df.date_1))\
.withColumn("week_number", weekofyear(df.date_1))\
.withColumn("day_of_year", dayofyear(df.date_1))\
.withColumn("day_of_week", dayofweek(df.date_1))

df_result.display()


date_str_1,date_str_2,date_1,date_2,day_of_month,week_number,day_of_year,day_of_week
2023-05-18,01 Jan 2010,2023-05-18,2010-01-01,18,20,138,5
2023-12-31,01 Jan 2010,2023-12-31,2010-01-01,31,52,365,1


#22. How to convert year-month string to dates corresponding to the 4th day of the month?

In [0]:
# example dataframe
df = spark.createDataFrame([('Jan 2010',), ('Feb 2011',), ('Mar 2012',)], ['MonthYear'])

df.show()

+---------+
MonthYear|
+---------+
 Jan 2010|
 Feb 2011|
 Mar 2012|
+---------+

In [0]:
### convert year-month string to dates corresponding to the 4th day of the month
from pyspark.sql.functions import concat,col,lit
df_result = df.withColumn("Date", to_date(col("MonthYear"),"MMM yyyy") +3)
df_result.display()

MonthYear,Date
Jan 2010,2010-01-04
Feb 2011,2011-02-04
Mar 2012,2012-03-04


#23 How to filter words that contain atleast 2 vowels from a series?

In [0]:
# example dataframe
df = spark.createDataFrame([('Apple',), ('Orange',), ('Plan',) , ('Python',) , ('Money',)], ['Word'])

df.show()

+------+
 Word|
+------+
 Apple|
Orange|
 Plan|
Python|
 Money|
+------+

In [0]:
### filter words that contain atleast 2 vowels from a series
from pyspark.sql.functions import expr, regexp_count
from pyspark.sql.functions import *
df_result = df.filter(expr("regexp_count(upper(Word),'[AEIOU]')") >=2)

# Define a custom function to check if a word has at least 2 vowels
def has_at_least_two_vowels(word):
    return expr("regexp_count(upper({}), '[AEIOU]') >= 2".format(word))

# Filter words with at least 2 vowels
df_result = df.filter(has_at_least_two_vowels("Word"))

df_result.display()


--------------------------------------------------------------------------- 
 ImportError Traceback (most recent call last)
 <command-3406428054828030> in <module> 
 1 ### filter words that contain atleast 2 vowels from a series 
 ----> 2 from pyspark . sql . functions import expr , regexp_count
 3 from pyspark . sql . functions import * 
 4 df_result = df . filter ( expr ( "regexp_count(upper(Word),'[AEIOU]')" ) >= 2 ) 
 5 

 ImportError : cannot import name 'regexp_count' from 'pyspark.sql.functions' (/databricks/spark/python/pyspark/sql/functions.py)

#24. How to filter valid emails from a list?

In [0]:
# Create a list
data = ['buying books at amazom.com', 'rameses@egypt.com', 'matt@t.co', 'narendra@modi.com']

# Convert the list to DataFrame
df = spark.createDataFrame(data, "string")
df.show(truncate =False)

+--------------------------+
value |
+--------------------------+
buying books at amazom.com|
rameses@egypt.com |
matt@t.co |
narendra@modi.com |
+--------------------------+

In [0]:
### filter valid emails from a list
df_result = df.filter(col("value").rlike(".+@.+.com"))
df_result.display()

pattern = "^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"

df_result = df.filter(col("value").rlike(pattern))
df_result.display()


value
rameses@egypt.com
narendra@modi.com


value
rameses@egypt.com
matt@t.co
narendra@modi.com


#25. How to Pivot PySpark DataFrame?
- Convert region categories to Columns and sum the revenue

In [0]:
# Sample data
data = [
(2021, 1, "US", 5000),
(2021, 1, "EU", 4000),
(2021, 2, "US", 5500),
(2021, 2, "EU", 4500),
(2021, 3, "US", 6000),
(2021, 3, "EU", 5000),
(2021, 4, "US", 7000),
(2021, 4, "EU", 6000),
]

# Create DataFrame
columns = ["year", "quarter", "region", "revenue"]
df = spark.createDataFrame(data, columns)
df.show()

+----+-------+------+-------+
year|quarter|region|revenue|
+----+-------+------+-------+
2021| 1| US| 5000|
2021| 1| EU| 4000|
2021| 2| US| 5500|
2021| 2| EU| 4500|
2021| 3| US| 6000|
2021| 3| EU| 5000|
2021| 4| US| 7000|
2021| 4| EU| 6000|
+----+-------+------+-------+

In [0]:
### Pivot PySpark DataFrame
df_result = df.groupBy("year","quarter").pivot("region").sum("revenue")
df_result.display()

year,quarter,EU,US
2021,2,4500,5500
2021,1,4000,5000
2021,3,5000,6000
2021,4,6000,7000


#26. How to get the mean of a variable grouped by another variable?

In [0]:
# Sample data
data = [("1001", "Laptop", 1000),
("1002", "Mouse", 50),
("1003", "Laptop", 1200),
("1004", "Mouse", 30),
("1005", "Smartphone", 700)]

# Create DataFrame
columns = ["OrderID", "Product", "Price"]
df = spark.createDataFrame(data, columns)

df.show()

+-------+----------+-----+
OrderID| Product|Price|
+-------+----------+-----+
 1001| Laptop| 1000|
 1002| Mouse| 50|
 1003| Laptop| 1200|
 1004| Mouse| 30|
 1005|Smartphone| 700|
+-------+----------+-----+

In [0]:
### mean of a variable grouped by another variable
from pyspark.sql.functions import mean
df_result = df.groupBy("Product").mean("Price").alias("Avg_Price")
df_result.display()
df_result = df.groupBy("Product").agg(mean("Price").alias("Avg_Price"))
df_result.display()

Product,avg(Price)
Laptop,1100.0
Mouse,40.0
Smartphone,700.0


Product,Avg_Price
Laptop,1100.0
Mouse,40.0
Smartphone,700.0


#27. How to compute the euclidean distance between two columns?
-Compute the euclidean distance between series (points) p and q, without using a packaged formula.

In [0]:
# Define your series
data = [(1, 10), (2, 9), (3, 8), (4, 7), (5, 6), (6, 5), (7, 4), (8, 3), (9, 2), (10, 1)]

# Convert list to DataFrame
df = spark.createDataFrame(data, ["series1", "series2"])

df.show()

+-------+-------+
series1|series2|
+-------+-------+
 1| 10|
 2| 9|
 3| 8|
 4| 7|
 5| 6|
 6| 5|
 7| 4|
 8| 3|
 9| 2|
 10| 1|
+-------+-------+

In [0]:
### uclidean distance between two columns
from pyspark.sql.functions import expr
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# Convert series to vectors
vecAssembler = VectorAssembler(inputCols=["series1", "series2"], outputCol="vectors")
df_tmp = vecAssembler.transform(df)


# Calculate squared differences
df_tmp = df_tmp.withColumn("squared_diff", expr("POW(series1 - series2, 2)"))
df_tmp.display()

# Sum squared differences and take square root
df_result = df_tmp.agg(expr("SQRT(SUM(squared_diff))").alias("euclidean_distance"))

df_result.display()

series1,series2,vectors,squared_diff
1,10,"Map(vectorType -> dense, length -> 2, values -> List(1.0, 10.0))",81.0
2,9,"Map(vectorType -> dense, length -> 2, values -> List(2.0, 9.0))",49.0
3,8,"Map(vectorType -> dense, length -> 2, values -> List(3.0, 8.0))",25.0
4,7,"Map(vectorType -> dense, length -> 2, values -> List(4.0, 7.0))",9.0
5,6,"Map(vectorType -> dense, length -> 2, values -> List(5.0, 6.0))",1.0
6,5,"Map(vectorType -> dense, length -> 2, values -> List(6.0, 5.0))",1.0
7,4,"Map(vectorType -> dense, length -> 2, values -> List(7.0, 4.0))",9.0
8,3,"Map(vectorType -> dense, length -> 2, values -> List(8.0, 3.0))",25.0
9,2,"Map(vectorType -> dense, length -> 2, values -> List(9.0, 2.0))",49.0
10,1,"Map(vectorType -> dense, length -> 2, values -> List(10.0, 1.0))",81.0


euclidean_distance
18.16590212458495


#28. How to replace missing spaces in a string with the least frequent character?

In [0]:
#Sample DataFrame
df = spark.createDataFrame([('dbc deb abed gade',),], ["string"])
df.show()

+-----------------+
 string|
+-----------------+
dbc deb abed gade|
+-----------------+

In [0]:
### replace missing spaces in a string with the least frequent character
from pyspark.sql.functions import regexp_replace,col,explode,split,count

df_tmp = df.withColumn("tmp1",regexp_replace(col("string"),'\\s+',''))
df_tmp = df_tmp.withColumn("tmp2",explode(split(col("tmp1"),'')))
df_tmp = df_tmp.groupBy("tmp2").agg(count("tmp2").alias("cnt"))

char_select = df_tmp.orderBy("cnt","tmp2").collect()[0][0]
print(char_select)

df_result = df.withColumn("tmp1",regexp_replace(col("string"),'\\s+',char_select))

df_result.display()

string,tmp1
dbc deb abed gade,dbcdebabedgade


In [0]:
from pyspark.sql.functions import udf, explode
from pyspark.sql.types import StringType, ArrayType
from collections import Counter
import statistics

def least_freq_char_replace_spaces(s):
    counter = Counter(s.replace(" ", ""))
    least_freq_char = statistics.mode(list(counter.elements()))
    return s.replace(' ', least_freq_char)

udf_least_freq_char_replace_spaces = udf(least_freq_char_replace_spaces, StringType())

df = spark.createDataFrame([('dbc deb abed gade',)], ["string"])
df.withColumn('modified_string', udf_least_freq_char_replace_spaces(df['string'])).show()

+-----------------+-----------------+
 string| modified_string|
+-----------------+-----------------+
dbc deb abed gade|dbcddebdabeddgade|
+-----------------+-----------------+

#29. How to create a TimeSeries starting ‘2000-01-01’ and 10 weekends (saturdays) after that having random numbers as values?

In [0]:
### TimeSeries starting ‘2000-01-01’ and 10 weekends (saturdays)
from pyspark.sql.functions import sequence,to_date,dayofweek,monotonically_increasing_id,randn,cast
from pyspark.sql.types import IntegerType
from pyspark.sql import Window
start_date = '2000-01-01'
end_date = '2024-12-31'

df = spark.sql(f"SELECT sequence(to_date('{start_date}'), to_date('{end_date}'), interval 1 day) as date")
df_tmp = df.withColumn("Dates",explode('date')).drop('date')
df_tmp = df_tmp.withColumn('DayWeek',dayofweek("Dates"))
df_tmp = df_tmp.filter(df_tmp["DayWeek"]==7)
df_tmp = df_tmp.withColumn("R",monotonically_increasing_id() +1)
df_tmp = df_tmp.filter(df_tmp["R"]<=10)

random_column = (randn(42) * 10 + 1).cast(IntegerType())

df_result = df_tmp.withColumn("Random",random_column).drop('DayWeek','R')


df_result.display()


--------------------------------------------------------------------------- 
 ImportError Traceback (most recent call last)
 <command-366566501270890> in <module> 
 1 ### TimeSeries starting ‘2000-01-01’ and 10 weekends (saturdays) 
 ----> 2 from pyspark . sql . functions import sequence , to_date , dayofweek , monotonically_increasing_id , randn , cast
 3 from pyspark . sql . types import IntegerType
 4 from pyspark . sql import Window
 5 start_date = '2000-01-01' 

 ImportError : cannot import name 'cast' from 'pyspark.sql.functions' (/databricks/spark/python/pyspark/sql/functions.py)

#30. How to get the nrows, ncolumns, datatype of a dataframe?
-Get the number of rows, columns, datatype and summary statistics of each column of the Churn_Modelling dataset. Also get the numpy array and list equivalent of the dataframe

In [0]:
from pyspark import SparkFiles

url = "https://raw.githubusercontent.com/selva86/datasets/master/Churn_Modelling.csv"

spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("Churn_Modelling.csv"), header=True, inferSchema=True)

#df = spark.read.csv("C:/Users/RajeshVaddi/Documents/MLPlus/DataSets/Churn_Modelling.csv", header=True, inferSchema=True)

df.show(5, truncate=False)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-366566501270896> in <module> 
 5 spark . sparkContext . addFile ( url ) 
 6 
 ----> 7 df = spark . read . csv ( SparkFiles . get ( "Churn_Modelling.csv" ) , header = True , inferSchema = True ) 
 8 
 9 #df = spark.read.csv("C:/Users/RajeshVaddi/Documents/MLPlus/DataSets/Churn_Modelling.csv", header=True, inferSchema=True) 

 /databricks/spark/python/pyspark/sql/readwriter.py in csv (self, path, schema, sep, encoding, quote, escape, comment, header, inferSchema, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, nullValue, nanValue, positiveInf, negativeInf, dateFormat, timestampFormat, maxColumns, maxCharsPerColumn, maxMalformedLogPerPartition, mode, columnNameOfCorruptRecord, multiLine, charToEscapeQuoteEscaping, samplingRatio, enforceSchema, emptyValue, locale, lineSep, pathGlobFilter, recursiveFileLookup, modifiedBefore, modifiedAfter, unescapedQuoteHandling) 
 762 path = [ path ] 
 763 if type ( path ) == list : 
 --> 764 return self . _df ( self . _jreader . csv ( self . _spark . _sc . _jvm . PythonUtils . toSeq ( path ) ) ) 
 765 elif isinstance ( path , RDD ) : 
 766 def func ( iterator ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Path does not exist: dbfs:/local_disk0/spark-719d73cc-2fd8-4de5-83a6-9bf822f9cef8/userFiles-27d14bd7-c017-4361-b8c7-7cedbcbaca9c/Churn_Modelling.csv

In [0]:
### the nrows, ncolumns, datatype of a dataframe
# For number of rows
nrows = df.count()
print("Number of Rows: ", nrows)

# For number of columns
ncols = len(df.columns)
print("Number of Columns: ", ncols)

# For data types of each column
datatypes = df.dtypes
print("Data types: ", datatypes)

Number of Rows: 1
Number of Columns: 1
Data types: [('string', 'string')]

#31. How to rename a specific columns in a dataframe?

In [0]:
# Suppose you have the following DataFrame
df = spark.createDataFrame([('Alice', 1, 30),('Bob', 2, 35)], ["name", "age", "qty"])

df.show()



+-----+---+---+
 name|age|qty|
+-----+---+---+
Alice| 1| 30|
 Bob| 2| 35|
+-----+---+---+

In [0]:
### rename a specific columns
# Rename lists for specific columns
old_names = ["qty", "age"]
new_names = ["user_qty", "user_age"]

for old,new in zip(old_names,new_names):
    df_result = df.withColumnRenamed(old,new)

df_result.display()

name,user_age,qty
Alice,1,30
Bob,2,35


#32. How to check if a dataframe has any missing values and count of missing values in each column?

In [0]:
# Assuming df is your DataFrame
df = spark.createDataFrame([
("A", 1, None),
("B", None, "123" ),
("B", 3, "456"),
("D", None, None),
], ["Name", "Value", "id"])

df.show()

+----+-----+----+
Name|Value| id|
+----+-----+----+
 A| 1|null|
 B| null| 123|
 B| 3| 456|
 D| null|null|
+----+-----+----+

In [0]:
### dataframe has any missing values and count of missing values
from pyspark.sql.functions import col, isnan, when, count

# Calculate the count of missing values for each column
df_result = df.select([count(when(isnan(c) | col(c).isNull() | (col(c) == ""), c)).alias(c) for c in df.columns])
df_result.display()



Name,Value,id
0,2,2


#33 How to replace missing values of multiple numeric columns with the mean?

In [0]:
df = spark.createDataFrame([
("A", 1, None),
("B", None, 123 ),
("B", 3, 456),
("D", 6, None),
], ["Name", "var1", "var2"])

df.show()

+----+----+----+
Name|var1|var2|
+----+----+----+
 A| 1|null|
 B|null| 123|
 B| 3| 456|
 D| 6|null|
+----+----+----+

In [0]:
### replace missing values of multiple numeric columns
from pyspark.sql.functions import *

df_tmp = df.withColumn('var1',when(col('var1').isNull(),df.groupBy().mean('var1').first()[0]).otherwise(col('var1')))\
    .withColumn('var2',when(col('var2').isNull(),df.groupBy().mean('var2').first()[0]).otherwise(col('var2')))
df_tmp.display()

Name,var1,var2
A,1.0,289.5
B,3.3333333333333335,123.0
B,3.0,456.0
D,6.0,289.5


In [0]:
### OR ML
from pyspark.ml.feature import Imputer

column_names = ["var1", "var2"]

# Initialize the Imputer
imputer = Imputer(inputCols= column_names, outputCols= column_names, strategy="mean")

# Fit the Imputer
model = imputer.fit(df)

#Transform the dataset
df_result = model.transform(df)

df_result.display()

Name,var1,var2
A,1,289
B,3,123
B,3,456
D,6,289


#34. How to change the order of columns of a dataframe?

In [0]:
# Sample data
data = [("John", "Doe", 30), ("Jane", "Doe", 25), ("Alice", "Smith", 22)]

# Create DataFrame from the data
df = spark.createDataFrame(data, ["First_Name", "Last_Name", "Age"])

# Show the DataFrame
df.show()

+----------+---------+---+
First_Name|Last_Name|Age|
+----------+---------+---+
 John| Doe| 30|
 Jane| Doe| 25|
 Alice| Smith| 22|
+----------+---------+---+

In [0]:
### change the order of columns
col_seq = ["Age", "First_Name", "Last_Name"]
df_result = df.select([*col_seq])
df_result.display()

Age,First_Name,Last_Name
30,John,Doe
25,Jane,Doe
22,Alice,Smith


#35. How to format or suppress scientific notations in a PySpark DataFrame?

In [0]:
# Assuming you have a DataFrame df and the column you want to format is 'your_column'
df = spark.createDataFrame([(1, 0.000000123), (2, 0.000023456), (3, 0.000345678)], ["id", "your_column"])

df.show()

+---+-----------+
 id|your_column|
+---+-----------+
 1| 1.23E-7|
 2| 2.3456E-5|
 3| 3.45678E-4|
+---+-----------+

In [0]:
### format or suppress scientific notations
from pyspark.sql.functions import format_number

# Determine the number of decimal places you want
decimal_places = 10

df_result = df.withColumn("your_column", format_number("your_column", decimal_places))
df_result.display()

id,your_column
1,0.0000001230
2,0.0000234560
3,0.0003456780


#36. How to format all the values in a dataframe as percentages?

In [0]:
# Sample data
data = [(0.1, .08), (0.2, .06), (0.33, .02)]
df = spark.createDataFrame(data, ["numbers_1", "numbers_2"])

df.show()

+---------+---------+
numbers_1|numbers_2|
+---------+---------+
 0.1| 0.08|
 0.2| 0.06|
 0.33| 0.02|
+---------+---------+

In [0]:
### all the values in a dataframe as percentages
from pyspark.sql.functions import *
from pyspark.sql.types import *

def format_columns(column_names,datafr)-> DataFrame:
    for colName in col_names:
        datafr = datafr.withColumn(colName,concat((col(colName) * 100).cast(DecimalType(18,2)),lit('%')))
    return datafr

df_result = format_columns(df.columns,df)
df_result.display()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4363358011361189> in <module> 
 8 return datafr
 9 
 ---> 10 df_result = format_columns ( df . columns , df ) 
 11 df_result . display ( ) 

 <command-4363358011361189> in format_columns (column_names, datafr) 
 4 
 5 def format_columns ( column_names , datafr ) -> DataFrame : 
 ----> 6 for colName in col_names : 
 7 datafr = datafr . withColumn ( colName , concat ( ( col ( colName ) * 100 ) . cast ( DecimalType ( 18 , 2 ) ) , lit ( '%' ) ) ) 
 8 return datafr

 NameError : name 'col_names' is not defined

#37. How to filter every nth row in a dataframe?

In [0]:
# Sample data
data = [("Alice", 1), ("Bob", 2), ("Charlie", 3), ("Dave", 4), ("Eve", 5),
("Frank", 6), ("Grace", 7), ("Hannah", 8), ("Igor", 9), ("Jack", 10)]

# Create DataFrame
df = spark.createDataFrame(data, ["Name", "Number"])

df.show()

+-------+------+
 Name|Number|
+-------+------+
 Alice| 1|
 Bob| 2|
Charlie| 3|
 Dave| 4|
 Eve| 5|
 Frank| 6|
 Grace| 7|
 Hannah| 8|
 Igor| 9|
 Jack| 10|
+-------+------+

In [0]:
### filter every nth row in a dataframe
from pyspark.sql.functions import *
from pyspark.sql import Window

def _filter_nth(_position)-> DataFrame:
    w = Window().orderBy(monotonically_increasing_id())
    df_tmp = df.withColumn('nth',row_number().over(w))
    df_tmp = df_tmp.filter((col('nth') % lit(_position))==0)
    return df_tmp

df_result = _filter_nth(3)


#38 How to get the row number of the nth largest value in a column?

In [0]:
from pyspark.sql import Row

# Sample Data
data = [
Row(id=1, column1=5),
Row(id=2, column1=8),
Row(id=3, column1=12),
Row(id=4, column1=1),
Row(id=5, column1=15),
Row(id=6, column1=7),
]

df = spark.createDataFrame(data)
df.show()

+---+-------+
 id|column1|
+---+-------+
 1| 5|
 2| 8|
 3| 12|
 4| 1|
 5| 15|
 6| 7|
+---+-------+

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, row_number

window = Window.orderBy(desc("column1"))
df = df.withColumn("row_number", row_number().over(window))

n = 3 # We're interested in the 3rd largest value.
row = df.filter(df.row_number == n).first()

if row:
    print("Row number:", row.row_number)
    print("Column value:", row.column1)

Row number: 3
Column value: 8

In [0]:
### get the row number of the nth largest value
from pyspark.sql.functions import *
from pyspark.sql import Window

w = Window.orderBy('id')
df_tmp = df.withColumn('position',row_number().over(w))
#df_tmp.display()
position = 3

df_tmp = df_tmp.filter(col('position') <= position)
df_result = df_tmp.filter(col('column1') == df_tmp.select(max(col('column1'))).collect()[0][0])
df_result.display()

id,column1,row_number,position
3,12,2,3


#39. How to get the last n rows of a dataframe with row sum > 100?

In [0]:
# Sample data
data = [(10, 25, 70),
(40, 5, 20),
(70, 80, 100),
(10, 2, 60),
(40, 50, 20)]

# Create DataFrame
df = spark.createDataFrame(data, ["col1", "col2", "col3"])

# Display original DataFrame
df.show()

+----+----+----+
col1|col2|col3|
+----+----+----+
 10| 25| 70|
 40| 5| 20|
 70| 80| 100|
 10| 2| 60|
 40| 50| 20|
+----+----+----+

In [0]:
### last n rows of a dataframe with row sum > 100
df_tmp = df.withColumn('summ',col('col1')+col('col2')+col('col3'))
df_tmp.display()
n=3
df_result_tmp = df_tmp.filter(col('summ') >= 100).tail(n)
df_result = spark.createDataFrame(df_result_tmp)
df_result.display()

col1,col2,col3,summ
10,25,70,105
40,5,20,65
70,80,100,250
10,2,60,72
40,50,20,110


col1,col2,col3,summ
10,25,70,105
70,80,100,250
40,50,20,110


#40. How to create a column containing the minimum by maximum of each row?

In [0]:
# Sample Data
data = [(1, 2, 3), (4, 5, 6), (7, 8, 9), (10, 11, 12)]

# Create DataFrame
df = spark.createDataFrame(data, ["col1", "col2", "col3"])

df.show()

+----+----+----+
col1|col2|col3|
+----+----+----+
 1| 2| 3|
 4| 5| 6|
 7| 8| 9|
 10| 11| 12|
+----+----+----+

In [0]:
### minimum by maximum of each row
from pyspark.sql.functions import *

df_tmp = df.withColumn("min_value", least(col("col1"),col("col2"),col("col3")))
df_tmp = df_tmp.withColumn("max_value", greatest(col("col1"),col("col2"),col("col3")))

# Create a new column with the ratio of min_value to max_value
df_result = df_tmp.withColumn("min_max_ratio", col("min_value") / col("max_value"))
df_result.display()

col1,col2,col3,min_value,max_value,min_max_ratio
1,2,3,1,3,0.3333333333333333
4,5,6,4,6,0.6666666666666666
7,8,9,7,9,0.7777777777777778
10,11,12,10,12,0.8333333333333334


#41. How to create a column that contains the penultimate value in each row?
- Create a new column ‘penultimate’ which has the second largest value of each row of df

In [0]:
data = [(10, 20, 30),
(40, 60, 50),
(80, 70, 90)]

df = spark.createDataFrame(data, ["Column1", "Column2", "Column3"])

df.show()

+-------+-------+-------+
Column1|Column2|Column3|
+-------+-------+-------+
 10| 20| 30|
 40| 60| 50|
 80| 70| 90|
+-------+-------+-------+

In [0]:
df_tmp = df.withColumn('my_array',sort_array(array(df.columns)))
df_result = df_tmp.withColumn('penultimate',col('my_array').getItem(1)).drop('my_array')
df_result.display()

Column1,Column2,Column3,penultimate
10,20,30,20
40,60,50,50
80,70,90,80


#42. How to normalize all columns in a dataframe?
- Normalize all columns of df by subtracting the column mean and divide by standard deviation.
- Range all columns of df such that the minimum value in each column is 0 and max is 1.

In [0]:
# create a sample dataframe
data = [(1, 2, 3),
(2, 3, 4),
(3, 4, 5),
(4, 5, 6)]

df = spark.createDataFrame(data, ["Col1", "Col2", "Col3"])

df.show()

+----+----+----+
Col1|Col2|Col3|
+----+----+----+
 1| 2| 3|
 2| 3| 4|
 3| 4| 5|
 4| 5| 6|
+----+----+----+

In [0]:
### normalize all columns
df_src=df

df_tmp = df.select(*[(mean(col(c)).alias(f'{c}_mean')) for c in df_src.columns],
                  *[(stddev(col(c)).alias(f'{c}_stddev')) for c in df_src.columns])

# Normalize columns using Z-score (standardization)
for c in df_src.columns:
    df_src = df_src.withColumn(c, (col(c) - df_tmp.first()[f"{c}_mean"]) / df_tmp.first()[f"{c}_stddev"])

# Min-max scaling
#for c in df.columns:
#    min_val = df_src.selectExpr(f"min({c})").first()[0]
#    max_val = df_src.selectExpr(f"max({c})").first()[0]
#    df_src = df_src.withColumn(c, (col(c) - min_val) / (max_val - min_val))

df_src.display()

Col1,Col2,Col3
-1.161895003862225,-1.161895003862225,-1.161895003862225
-0.3872983346207417,-0.3872983346207417,-0.3872983346207417
0.3872983346207417,0.3872983346207417,0.3872983346207417
1.161895003862225,1.161895003862225,1.161895003862225


#43. How to get the positions where values of two columns match?

In [0]:
# Create sample DataFrame
data = [("John", "John"), ("Lily", "Lucy"), ("Sam", "Sam"), ("Lucy", "Lily")]
df = spark.createDataFrame(data, ["Name1", "Name2"])

df.show()

+-----+-----+
Name1|Name2|
+-----+-----+
 John| John|
 Lily| Lucy|
 Sam| Sam|
 Lucy| Lily|
+-----+-----+

In [0]:
df_result = df.withColumn('Match',when(col('Name1')==col('Name2'),True).otherwise(False))
df_result.display()

Name1,Name2,Match
John,John,true
Lily,Lucy,false
Sam,Sam,true
Lucy,Lily,false


#44. How to create lags and leads of a column by group in a dataframe?

In [0]:
# Create a sample DataFrame
data = [("2023-01-01", "Store1", 100),
("2023-01-02", "Store1", 150),
("2023-01-03", "Store1", 200),
("2023-01-04", "Store1", 250),
("2023-01-05", "Store1", 300),
("2023-01-01", "Store2", 50),
("2023-01-02", "Store2", 60),
("2023-01-03", "Store2", 80),
("2023-01-04", "Store2", 90),
("2023-01-05", "Store2", 120)]

df = spark.createDataFrame(data, ["Date", "Store", "Sales"])

df.show()

+----------+------+-----+
 Date| Store|Sales|
+----------+------+-----+
2023-01-01|Store1| 100|
2023-01-02|Store1| 150|
2023-01-03|Store1| 200|
2023-01-04|Store1| 250|
2023-01-05|Store1| 300|
2023-01-01|Store2| 50|
2023-01-02|Store2| 60|
2023-01-03|Store2| 80|
2023-01-04|Store2| 90|
2023-01-05|Store2| 120|
+----------+------+-----+

In [0]:
### lags and leads of a column by group
from pyspark.sql.functions import *
from pyspark.sql.window import Window

# Convert the date from string to date type
df_tmp = df.withColumn("Date", to_date(df.Date, 'yyyy-MM-dd'))

# Create a Window partitioned by Store, ordered by Date
windowSpec = Window.partitionBy("Store").orderBy("Date")

# Create lag and lead variables
df_tmp = df_tmp.withColumn("Lag_Sales", lag(df_tmp["Sales"]).over(windowSpec))
df_result = df_tmp.withColumn("Lead_Sales", lead(df_tmp["Sales"]).over(windowSpec))

df_result.display()

Date,Store,Sales,Lag_Sales,Lead_Sales
2023-01-01,Store1,100,null,150
2023-01-02,Store1,150,100,200
2023-01-03,Store1,200,150,250
2023-01-04,Store1,250,200,300
2023-01-05,Store1,300,250,null
2023-01-01,Store2,50,null,60
2023-01-02,Store2,60,50,80
2023-01-03,Store2,80,60,90
2023-01-04,Store2,90,80,120
2023-01-05,Store2,120,90,null


#45. How to get the frequency of unique values in the entire dataframe?

In [0]:
# Create a numeric DataFrame
data = [(1, 2, 3),
(2, 3, 4),
(1, 2, 3),
(4, 5, 6),
(2, 3, 4)]
df = spark.createDataFrame(data, ["Column1", "Column2", "Column3"])

# Print DataFrame
df.show()

+-------+-------+-------+
Column1|Column2|Column3|
+-------+-------+-------+
 1| 2| 3|
 2| 3| 4|
 1| 2| 3|
 4| 5| 6|
 2| 3| 4|
+-------+-------+-------+

In [0]:
### frequency of unique values in the entire dataframe
from pyspark.sql.functions import *

df_result = None

for c in df.columns:
    if df_result is None:
        df_result = df.select(col(c).alias('value'))
    else:
        df_result = df_result.union(df.select(col(c).alias('value')))

df_result = df_result.groupBy(col('value')).agg(count(col('value')).alias('freq')).orderBy(col('value'))
df_result.display()



value,freq
1,2
2,4
3,4
4,3
5,1
6,1


#46. How to replace both the diagonals of dataframe with 0?

In [0]:
# Create a numeric DataFrame
data = [(1, 2, 3, 4),
(2, 3, 4, 5),
(1, 2, 3, 4),
(4, 5, 6, 7)]

df = spark.createDataFrame(data, ["col_1", "col_2", "col_3", "col_4"])

# Print DataFrame
df.show()

+-----+-----+-----+-----+
col_1|col_2|col_3|col_4|
+-----+-----+-----+-----+
 1| 2| 3| 4|
 2| 3| 4| 5|
 1| 2| 3| 4|
 4| 5| 6| 7|
+-----+-----+-----+-----+

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql.functions import when, col

# Define window specification
w = Window.orderBy(monotonically_increasing_id())

# Add index
df = df.withColumn("id", row_number().over(w) - 1)

df = df.select([when(col("id") == i, 0).otherwise(col("col_"+str(i+1))).alias("col_"+str(i+1)) for i in range(4)])

# Create a reverse id column
df = df.withColumn("id", row_number().over(w) - 1)
df = df.withColumn("id_2", df.count() - 1 - df["id"])

df_with_diag_zero = df.select([when(col("id_2") == i, 0).otherwise(col("col_"+str(i+1))).alias("col_"+str(i+1)) for i in range(4)])

df_with_diag_zero.show()

+-----+-----+-----+-----+
col_1|col_2|col_3|col_4|
+-----+-----+-----+-----+
 0| 2| 3| 0|
 2| 0| 0| 5|
 1| 0| 0| 4|
 0| 5| 6| 0|
+-----+-----+-----+-----+

In [0]:
from pyspark.sql.functions import *
from pyspark.sql import *

w = Window.orderBy(monotonically_increasing_id())
df_tmp = df.withColumn('nrow',row_number().over(w))
ncols=len(df.columns)
nlins=df.count()
for i in range(0,ncls):
    df_tmp = df_tmp.select()


df_tmp.display()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1459838748039776> in <module> 
 6 ncols = len ( df . columns ) 
 7 nlins = df . count ( ) 
 ----> 8 for i in range ( 0 , ncls ) : 
 9 df_tmp = df_tmp . select ( ) 
 10 

 NameError : name 'ncls' is not defined

#47. How to reverse the rows of a dataframe?

In [0]:
# Create a numeric DataFrame
data = [(1, 2, 3, 4),
(2, 3, 4, 5),
(3, 4, 5, 6),
(4, 5, 6, 7)]

df = spark.createDataFrame(data, ["col_1", "col_2", "col_3", "col_4"])

# Print DataFrame
df.show()

+-----+-----+-----+-----+
col_1|col_2|col_3|col_4|
+-----+-----+-----+-----+
 1| 2| 3| 4|
 2| 3| 4| 5|
 3| 4| 5| 6|
 4| 5| 6| 7|
+-----+-----+-----+-----+

In [0]:
from pyspark.sql.functions import *
from pyspark.sql import Window

w = Window.orderBy(monotonically_increasing_id())

df_tmp = df.withColumn('row',row_number().over(w))
df_result = df_tmp.orderBy('row',ascending=False).drop('row')

df_result.display()


col_1,col_2,col_3,col_4
4,5,6,7
3,4,5,6
2,3,4,5
1,2,3,4


#48. How to create one-hot encodings of a categorical variable (dummy variables)?
- Get one-hot encodings for column Categories in the dataframe df and append it as columns.

In [0]:
data = [("A", 10),("A", 20),("B", 30),("B", 20),("B", 30),("C", 40),("C", 10),("D", 10)]
#data = [("A", 10),("A", 20),("B", 30),("B", 20),("B", 30),("C", 40),("C", 10)]
data = [("A", 10),("A", 20),("B", 30)]
columns = ["Categories", "Value"]

df = spark.createDataFrame(data, columns)
df.show()

+----------+-----+
Categories|Value|
+----------+-----+
 A| 10|
 A| 20|
 B| 30|
+----------+-----+

In [0]:
### one-hot encodings of a categorical variable
from pyspark.ml.feature import StringIndexer, OneHotEncoder
#from pyspark.sql.types import StringType, StructType, StructField

# StringIndexer Initialization
indexer = StringIndexer(inputCol="Categories", outputCol="Categories_Indexed")
indexerModel = indexer.fit(df)

# Transform the DataFrame using the fitted StringIndexer model
indexed_df = indexerModel.transform(df)
#indexed_df.show()

encoder = OneHotEncoder(inputCol="Categories_Indexed", outputCol="Categories_onehot")
encoded_df = encoder.fit(indexed_df).transform(indexed_df)
#encoded_df = encoded_df.drop("Categories_Indexed")
encoded_df.show(truncate=False)

+----------+-----+------------------+-----------------+
Categories|Value|Categories_Indexed|Categories_onehot|
+----------+-----+------------------+-----------------+
A |10 |0.0 |(1,[0],[1.0]) |
A |20 |0.0 |(1,[0],[1.0]) |
B |30 |1.0 |(1,[],[]) |
+----------+-----+------------------+-----------------+

#49. How to Pivot the dataframe (converting rows into columns) ?
- convert region column categories to Column

In [0]:
# Sample data
data = [
(2021, 1, "US", 5000),
(2021, 1, "EU", 4000),
(2021, 2, "US", 5500),
(2021, 2, "EU", 4500),
(2021, 3, "US", 6000),
(2021, 3, "EU", 5000),
(2021, 4, "US", 7000),
(2021, 4, "EU", 6000),
]

# Create DataFrame
columns = ["year", "quarter", "region", "revenue"]
df = spark.createDataFrame(data, columns)
df.show()

+----+-------+------+-------+
year|quarter|region|revenue|
+----+-------+------+-------+
2021| 1| US| 5000|
2021| 1| EU| 4000|
2021| 2| US| 5500|
2021| 2| EU| 4500|
2021| 3| US| 6000|
2021| 3| EU| 5000|
2021| 4| US| 7000|
2021| 4| EU| 6000|
+----+-------+------+-------+

In [0]:
from pyspark.sql.functions import *
from pyspark.sql import *

df_result = df.groupBy('year','quarter').pivot('region').sum('revenue')
df_result.display()

year,quarter,EU,US
2021,2,4500,5500
2021,1,4000,5000
2021,3,5000,6000
2021,4,6000,7000


#50. How to UnPivot the dataframe (converting columns into rows) ?
- UnPivot EU, US columns and create region, revenue Columns

In [0]:
# Sample data
data = [(2021, 2, 4500, 5500),
(2021, 1, 4000, 5000),
(2021, 3, 5000, 6000),
(2021, 4, 6000, 7000)]

# Create DataFrame
columns = ["year", "quarter", "EU", "US"]
df = spark.createDataFrame(data, columns)

df.show()

+----+-------+----+----+
year|quarter| EU| US|
+----+-------+----+----+
2021| 2|4500|5500|
2021| 1|4000|5000|
2021| 3|5000|6000|
2021| 4|6000|7000|
+----+-------+----+----+

In [0]:


### UnPivot the dataframe
df_result = df.unpivot(ids=["year", "quarter"],
                          values=["EU", "US"],
                          variableColumnName="country",
                          valueColumnName="value")

df_result.display()


--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-607835847313252> in <module> 
 1 ### UnPivot the dataframe 
 ----> 2 df_result = df.unpivot(ids=["year", "quarter"],
 3 values = [ "EU" , "US" ] , 
 4 variableColumnName = "country" , 
 5 valueColumnName="value")

 /databricks/spark/python/pyspark/sql/dataframe.py in __getattr__ (self, name) 
 1664 """
 1665 if name not in self . columns : 
 -> 1666 raise AttributeError(
 1667 "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
 1668 jc = self . _jdf . apply ( name ) 

 AttributeError : 'DataFrame' object has no attribute 'unpivot'

#51. How to impute missing values with Zero?

In [0]:
# Suppose df is your DataFrame
df = spark.createDataFrame([(1, None), (None, 2), (3, 4), (5, None)], ["a", "b"])

df.show()

+----+----+
 a| b|
+----+----+
 1|null|
null| 2|
 3| 4|
 5|null|
+----+----+

In [0]:
df_result = df.select(*[when(col(c).isNull(),lit(0)).otherwise(col(c)).alias(c) for c in df.columns])
df_result.display()

a,b
1,0
0,2
3,4
5,0


In [0]:
df_imputed = df.fillna(0)

df_imputed.show()

+---+---+
 a| b|
+---+---+
 1| 0|
 0| 2|
 3| 4|
 5| 0|
+---+---+

#52. How to identify continuous variables in a dataframe and create a list of those column names?

In [0]:
url = "https://raw.githubusercontent.com/selva86/datasets/master/Churn_Modelling_m.csv"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("Churn_Modelling_m.csv"), header=True, inferSchema=True)

#df = spark.read.csv("C:/Users/RajeshVaddi/Documents/MLPlus/DataSets/Churn_Modelling_m.csv", header=True, inferSchema=True)

df.show(2, truncate=False)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-607835847313258> in <module> 
 2 spark . sparkContext . addFile ( url ) 
 3 
 ----> 4 df = spark . read . csv ( SparkFiles . get ( "Churn_Modelling_m.csv" ) , header = True , inferSchema = True ) 
 5 
 6 #df = spark.read.csv("C:/Users/RajeshVaddi/Documents/MLPlus/DataSets/Churn_Modelling_m.csv", header=True, inferSchema=True) 

 /databricks/spark/python/pyspark/sql/readwriter.py in csv (self, path, schema, sep, encoding, quote, escape, comment, header, inferSchema, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, nullValue, nanValue, positiveInf, negativeInf, dateFormat, timestampFormat, maxColumns, maxCharsPerColumn, maxMalformedLogPerPartition, mode, columnNameOfCorruptRecord, multiLine, charToEscapeQuoteEscaping, samplingRatio, enforceSchema, emptyValue, locale, lineSep, pathGlobFilter, recursiveFileLookup, modifiedBefore, modifiedAfter, unescapedQuoteHandling) 
 762 path = [ path ] 
 763 if type ( path ) == list : 
 --> 764 return self . _df ( self . _jreader . csv ( self . _spark . _sc . _jvm . PythonUtils . toSeq ( path ) ) ) 
 765 elif isinstance ( path , RDD ) : 
 766 def func ( iterator ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Path does not exist: dbfs:/local_disk0/spark-719d73cc-2fd8-4de5-83a6-9bf822f9cef8/userFiles-27d14bd7-c017-4361-b8c7-7cedbcbaca9c/Churn_Modelling_m.csv

In [0]:
from pyspark.sql.types import IntegerType, StringType, NumericType
from pyspark.sql.functions import approxCountDistinct

def detect_continuous_variables(df, distinct_threshold):
    """
    Identify continuous variables in a PySpark DataFrame.
    :param df: The input PySpark DataFrame
    :param distinct_threshold: Threshold to qualify as continuous variables - Count of distinct values > distinct_threshold
    :return: A List containing names of continuous variables
    """
    continuous_columns = []
    for column in df.columns:
        dtype = df.schema[column].dataType
        if isinstance(dtype, (IntegerType, NumericType)):
            distinct_count = df.select(approxCountDistinct(column)).collect()[0][0]
        if distinct_count > distinct_threshold:
            continuous_columns.append(column)
        return continuous_columns

continuous_variables = detect_continuous_variables(df, 10)
print(continuous_variables)
#['RowNumber', 'CustomerId', 'CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary']

[]

#53. How to calculate Mode of a PySpark DataFrame column?

In [0]:
# Create a sample DataFrame
data = [(1, 2, 3), (2, 2, 3), (2, 2, 4), (1, 2, 3), (1, 1, 3)]
columns = ["col1", "col2", "col3"]

df = spark.createDataFrame(data, columns)

df.show()

+----+----+----+
col1|col2|col3|
+----+----+----+
 1| 2| 3|
 2| 2| 3|
 2| 2| 4|
 1| 2| 3|
 1| 1| 3|
+----+----+----+

In [0]:
from pyspark.sql.functions import col

df_grouped = df.groupBy('col2').count()
mode_df = df_grouped.orderBy(col('count').desc()).limit(1)

mode_df.show()

+----+-----+
col2|count|
+----+-----+
 2| 4|
+----+-----+

#54. How to find installed location of Apache Spark and PySpark?

In [0]:
### find installed location of Apache Spark and PySpark
import findspark
findspark.init()

print(findspark.find())

import os
import pyspark

print(os.path.dirname(pyspark.__file__))

--------------------------------------------------------------------------- 
 ModuleNotFoundError Traceback (most recent call last)
 <command-3151542764839094> in <module> 
 1 ### find installed location of Apache Spark and PySpark 
 ----> 2 import findspark
 3 findspark . init ( ) 
 4 
 5 print ( findspark . find ( ) ) 

 /databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py in import_patch (name, globals, locals, fromlist, level) 
 160 # Import the desired module. If you’re seeing this while debugging a failed import, 
 161 # look at preceding stack frames for relevant error information. 
 --> 162 original_result = python_builtin_import ( name , globals , locals , fromlist , level ) 
 163 
 164 is_root_import = thread_local . _nest_level == 1 

 ModuleNotFoundError : No module named 'findspark'

#55. How to convert a column to lower case using UDF?

In [0]:
# Create a DataFrame to test
data = [('John Doe', 'NEW YORK'),
('Jane Doe', 'LOS ANGELES'),
('Mike Johnson', 'CHICAGO'),
('Sara Smith', 'SAN FRANCISCO')]

df = spark.createDataFrame(data, ['Name', 'City'])

df.show()

+------------+-------------+
 Name| City|
+------------+-------------+
 John Doe| NEW YORK|
 Jane Doe| LOS ANGELES|
Mike Johnson| CHICAGO|
 Sara Smith|SAN FRANCISCO|
+------------+-------------+

In [0]:

def lower_case(col_name,data_frame) -> DataFrame:
    df_tmp = data_frame.withColumn(col_name,lower(col(col_name)))
    return df_tmp

df_result = lower_case('Name',df)
df_result.display()


def lower_case(data_frame) -> DataFrame:
    df_tmp = data_frame.select(*[lower(col(c)).alias(c) for c in df.columns])
    return df_tmp

df_result = lower_case(df)
df_result.display()


Name,City
john doe,NEW YORK
jane doe,LOS ANGELES
mike johnson,CHICAGO
sara smith,SAN FRANCISCO


Name,City
john doe,new york
jane doe,los angeles
mike johnson,chicago
sara smith,san francisco


In [0]:
### convert a column to lower case using UDF
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Define your UDF function
def to_lower(s):
    if s is not None:
        return s.lower()

# Convert your Python function to a Spark UDF
udf_to_lower = udf(to_lower, StringType())

# Apply your UDF to the DataFrame
df = df.withColumn('City_lower', udf_to_lower(df['City']))

# Show the DataFrame
df.display()

Name,City,City_lower
John Doe,NEW YORK,new york
Jane Doe,LOS ANGELES,los angeles
Mike Johnson,CHICAGO,chicago
Sara Smith,SAN FRANCISCO,san francisco


#56. How to convert PySpark data frame to pandas dataframe?

In [0]:
# Create a DataFrame to test
data = [('John Doe', 'NEW YORK'),
('Jane Doe', 'LOS ANGELES'),
('Mike Johnson', 'CHICAGO'),
('Sara Smith', 'SAN FRANCISCO')]

pysparkDF = spark.createDataFrame(data, ['Name', 'City'])

pysparkDF.show()

+------------+-------------+
 Name| City|
+------------+-------------+
 John Doe| NEW YORK|
 Jane Doe| LOS ANGELES|
Mike Johnson| CHICAGO|
 Sara Smith|SAN FRANCISCO|
+------------+-------------+

In [0]:
df_result = pysparkDF.toPandas()
print(df_result)

Name City
0 John Doe NEW YORK
1 Jane Doe LOS ANGELES
2 Mike Johnson CHICAGO
3 Sara Smith SAN FRANCISCO

#57. How to View PySpark Cluster Details?

In [0]:
### Cluster Details
print(spark.sparkContext.uiWebUrl)

http://172.24.129.196:40001

#58. How to View PySpark Cluster Configuration Details?

In [0]:
### Cluster Details
print(spark.sparkContext.uiWebUrl)
# Print all configurations
for k,v in spark.sparkContext.getConf().getAll():
    print(f"{k} : {v}")

http://172.24.129.196:40001
spark.databricks.preemption.enabled : true
spark.databricks.clusterUsageTags.driverContainerId : ae4932b93ce24c0da2d5cad7243bc496
fs.azure.account.oauth2.client.secret.qastdeveuwdlsdsl01.dfs.core.windows.net : [REDACTED]
spark.databricks.clusterUsageTags.clusterFirstOnDemand : 1
spark.driver.tempDirectory : /local_disk0/tmp
spark.databricks.queryWatchdog.maxQueryTasks : 200000
spark.databricks.managedCatalog.clientClassName : com.databricks.managedcatalog.ManagedCatalogClientImpl
spark.databricks.clusterUsageTags.sparkImageLabel : release__9.1.x-snapshot-scala2.12__databricks-universe__9.1.61__73212e5__c999d52__jenkins__0f87a2f__format-3
spark.hadoop.fs.fcfs-s3.impl.disable.cache : true
spark.sql.streaming.checkpointFileManagerClass : com.databricks.spark.sql.streaming.DatabricksCheckpointFileManager
spark.databricks.service.dbutils.repl.backend : com.databricks.dbconnect.ReplDBUtils
spark.streaming.driver.writeAheadLog.allowBatching : true
spark.hadoop.hive.server2.transport.mode : http
spark.databricks.clusterUsageTags.driverInstanceId : 1b8c43f2ed3f482ba4ac1aa290277907
fs.azure.account.oauth.provider.type.datafounddeveuwdlsiss.dfs.core.windows.net : org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider
spark.app.id : app-20240510174946-0000
spark.hadoop.fs.cpfs-adl.impl.disable.cache : true
spark.databricks.clusterUsageTags.hailEnabled : false
spark.hadoop.fs.mcfs-s3.impl : com.databricks.sql.acl.fs.ManagedCatalogFileSystem
spark.databricks.clusterUsageTags.containerType : LXC
spark.eventLog.enabled : false
spark.databricks.clusterUsageTags.isIMv2Enabled : false
spark.databricks.cluster.profile : serverless
spark.repl.class.outputDir : /local_disk0/tmp/repl/spark-4618732084335514242-f8147b84-3151-4f4f-b891-a1eea072ef71
spark.hadoop.hive.hmshandler.retry.interval : 2000
spark.executor.tempDirectory : /local_disk0/tmp
spark.databricks.clusterUsageTags.clusterLastActivityTime : 1715359672603
spark.databricks.secret.sparkConf.keys.toRedact : ZnMuYXp1cmUuYWNjb3VudC5vYXV0aDIuY2xpZW50LnNlY3JldC5xYXN0ZGV2ZXV3ZGxzZHNsMDEuZGZzLmNvcmUud2luZG93cy5uZXQ=,ZnMuYXp1cmUuYWNjb3VudC5vYXV0aDIuY2xpZW50LmlkLmRhdGFmb3VuZGRldmV1d2Rsc2lzcy5kZnMuY29yZS53aW5kb3dzLm5ldA==,ZnMuYXp1cmUuYWNjb3VudC5vYXV0aDIuY2xpZW50LnNlY3JldC5kYXRhZm91bmRkZXZldXdkbHNpc3MuZGZzLmNvcmUud2luZG93cy5uZXQ=,ZnMuYXp1cmUuYWNjb3VudC5vYXV0aDIuY2xpZW50LmlkLnFhc3RkZXZldXdkbHNkc2wwMS5kZnMuY29yZS53aW5kb3dzLm5ldA==,ZnMuYXp1cmUuYWNjb3VudC5vYXV0aDIuY2xpZW50LnNlY3JldC5kYXRhZm91bmRkZXZldXdkbHNlbnIuZGZzLmNvcmUud2luZG93cy5uZXQ=,ZnMuYXp1cmUuYWNjb3VudC5vYXV0aDIuY2xpZW50LmlkLmRhdGFmb3VuZGRldmV1d2Rsc2Vuci5kZnMuY29yZS53aW5kb3dzLm5ldA==
spark.databricks.clusterUsageTags.clusterCreator : ThirdPartyApp
spark.hadoop.mapred.output.committer.class : com.databricks.backend.daemon.data.client.DirectOutputCommitter
spark.databricks.clusterUsageTags.clusterTargetWorkers : 2
spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version : 2
spark.databricks.clusterUsageTags.numPerClusterInitScriptsV2S3 : 0
spark.sql.allowMultipleContexts : false
spark.databricks.eventLog.enabled : true
spark.hadoop.hive.server2.thrift.http.port : 10000
spark.home : /databricks/spark
spark.hadoop.hive.server2.idle.operation.timeout : 7200000
fs.azure.account.oauth2.client.endpoint.qastdeveuwdlsdsl01.dfs.core.windows.net : https://login.microsoftonline.com/f25493ae-1c98-41d7-8a33-0be75f5fe603/oauth2/token
spark.repl.class.uri : spark://172.24.129.196:35411/classes
spark.task.reaper.enabled : true
spark.storage.memoryFraction : 0.5
spark.databricks.clusterUsageTags.autoTerminationMinutes : 60
spark.databricks.clusterUsageTags.sparkEnvVarContainsNewline : false
spark.hadoop.fs.fcfs-s3.impl : com.databricks.sql.acl.fs.FixedCredentialsFileSystem
spark.databricks.delta.multiClusterWrites.enabled : true
spark.databricks.clusterUsageTags.driverInstancePrivateIp : 172.24.129.132
spark.worker.cleanup.enabled : false
spark.sql.legacy.createHiveTableByDefault : false
spark.databricks.driver.preferredMavenCe

#59. How to restrict the PySpark to use the number of cores in the system?

In [0]:
### Cluster Details
print(spark.sparkContext.uiWebUrl)
# Print all configurations
for k,v in spark.sparkContext.getConf().getAll():
    print(f"{k} : {v}")
    
### PySpark to use the number of cores
from pyspark import SparkConf, SparkContext

conf = SparkConf()
conf.set("spark.executor.cores", "2") # set the number of cores you want here
sc = SparkContext(conf=conf)

http://172.24.129.196:40001
spark.databricks.preemption.enabled : true
spark.databricks.clusterUsageTags.driverContainerId : ae4932b93ce24c0da2d5cad7243bc496
fs.azure.account.oauth2.client.secret.qastdeveuwdlsdsl01.dfs.core.windows.net : [REDACTED]
spark.databricks.clusterUsageTags.clusterFirstOnDemand : 1
spark.driver.tempDirectory : /local_disk0/tmp
spark.databricks.queryWatchdog.maxQueryTasks : 200000
spark.databricks.managedCatalog.clientClassName : com.databricks.managedcatalog.ManagedCatalogClientImpl
spark.databricks.clusterUsageTags.sparkImageLabel : release__9.1.x-snapshot-scala2.12__databricks-universe__9.1.61__73212e5__c999d52__jenkins__0f87a2f__format-3
spark.hadoop.fs.fcfs-s3.impl.disable.cache : true
spark.sql.streaming.checkpointFileManagerClass : com.databricks.spark.sql.streaming.DatabricksCheckpointFileManager
spark.databricks.service.dbutils.repl.backend : com.databricks.dbconnect.ReplDBUtils
spark.streaming.driver.writeAheadLog.allowBatching : true
spark.hadoop.hive.server2.transport.mode : http
spark.databricks.clusterUsageTags.driverInstanceId : 1b8c43f2ed3f482ba4ac1aa290277907
fs.azure.account.oauth.provider.type.datafounddeveuwdlsiss.dfs.core.windows.net : org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider
spark.app.id : app-20240510174946-0000
spark.hadoop.fs.cpfs-adl.impl.disable.cache : true
spark.databricks.clusterUsageTags.hailEnabled : false
spark.hadoop.fs.mcfs-s3.impl : com.databricks.sql.acl.fs.ManagedCatalogFileSystem
spark.databricks.clusterUsageTags.containerType : LXC
spark.eventLog.enabled : false
spark.databricks.clusterUsageTags.isIMv2Enabled : false
spark.databricks.cluster.profile : serverless
spark.repl.class.outputDir : /local_disk0/tmp/repl/spark-4618732084335514242-f8147b84-3151-4f4f-b891-a1eea072ef71
spark.hadoop.hive.hmshandler.retry.interval : 2000
spark.executor.tempDirectory : /local_disk0/tmp
spark.databricks.clusterUsageTags.clusterLastActivityTime : 1715359672603
spark.databricks.secret.sparkConf.keys.toRedact : ZnMuYXp1cmUuYWNjb3VudC5vYXV0aDIuY2xpZW50LnNlY3JldC5xYXN0ZGV2ZXV3ZGxzZHNsMDEuZGZzLmNvcmUud2luZG93cy5uZXQ=,ZnMuYXp1cmUuYWNjb3VudC5vYXV0aDIuY2xpZW50LmlkLmRhdGFmb3VuZGRldmV1d2Rsc2lzcy5kZnMuY29yZS53aW5kb3dzLm5ldA==,ZnMuYXp1cmUuYWNjb3VudC5vYXV0aDIuY2xpZW50LnNlY3JldC5kYXRhZm91bmRkZXZldXdkbHNpc3MuZGZzLmNvcmUud2luZG93cy5uZXQ=,ZnMuYXp1cmUuYWNjb3VudC5vYXV0aDIuY2xpZW50LmlkLnFhc3RkZXZldXdkbHNkc2wwMS5kZnMuY29yZS53aW5kb3dzLm5ldA==,ZnMuYXp1cmUuYWNjb3VudC5vYXV0aDIuY2xpZW50LnNlY3JldC5kYXRhZm91bmRkZXZldXdkbHNlbnIuZGZzLmNvcmUud2luZG93cy5uZXQ=,ZnMuYXp1cmUuYWNjb3VudC5vYXV0aDIuY2xpZW50LmlkLmRhdGFmb3VuZGRldmV1d2Rsc2Vuci5kZnMuY29yZS53aW5kb3dzLm5ldA==
spark.databricks.clusterUsageTags.clusterCreator : ThirdPartyApp
spark.hadoop.mapred.output.committer.class : com.databricks.backend.daemon.data.client.DirectOutputCommitter
spark.databricks.clusterUsageTags.clusterTargetWorkers : 2
spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version : 2
spark.databricks.clusterUsageTags.numPerClusterInitScriptsV2S3 : 0
spark.sql.allowMultipleContexts : false
spark.databricks.eventLog.enabled : true
spark.hadoop.hive.server2.thrift.http.port : 10000
spark.home : /databricks/spark
spark.hadoop.hive.server2.idle.operation.timeout : 7200000
fs.azure.account.oauth2.client.endpoint.qastdeveuwdlsdsl01.dfs.core.windows.net : https://login.microsoftonline.com/f25493ae-1c98-41d7-8a33-0be75f5fe603/oauth2/token
spark.repl.class.uri : spark://172.24.129.196:35411/classes
spark.task.reaper.enabled : true
spark.storage.memoryFraction : 0.5
spark.databricks.clusterUsageTags.autoTerminationMinutes : 60
spark.databricks.clusterUsageTags.sparkEnvVarContainsNewline : false
spark.hadoop.fs.fcfs-s3.impl : com.databricks.sql.acl.fs.FixedCredentialsFileSystem
spark.databricks.delta.multiClusterWrites.enabled : true
spark.databricks.clusterUsageTags.driverInstancePrivateIp : 172.24.129.132
spark.worker.cleanup.enabled : false
spark.sql.legacy.createHiveTableByDefault : false
spark.databricks.driver.preferredMavenCe

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-3151542764839107> in <module> 
 10 conf = SparkConf ( ) 
 11 conf . set ( "spark.executor.cores" , "2" ) # set the number of cores you want here 
 ---> 12 sc = SparkContext ( conf = conf ) 

 /databricks/spark/python/pyspark/context.py in __init__ (self, master, appName, sparkHome, pyFiles, environment, batchSize, serializer, conf, gateway, jsc, profiler_cls) 
 143 " is not allowed as it is a security risk.")
 144 
 --> 145 SparkContext . _ensure_initialized ( self , gateway = gateway , conf = conf ) 
 146 try : 
 147 self._do_init(master, appName, sparkHome, pyFiles, environment, batchSize, serializer,

 /databricks/spark/python/pyspark/context.py in _ensure_initialized (cls, instance, gateway, conf) 
 348 
 349 # Raise error if there is already a running Spark context 
 --> 350 raise ValueError(
 351 "Cannot run multiple SparkContexts at once; " 
 352 "existing SparkContext(app=%s, master=%s)" 

 ValueError : Cannot run multiple SparkContexts at once; existing SparkContext(app=Databricks Shell, master=spark://172.24.129.196:7077) created by __init__ at /databricks/python_shell/scripts/PythonShellImpl.py:1223

#60. How to cache PySpark DataFrame or objects and delete cache?
- In PySpark, caching or persisting data is done to speed up data retrieval during iterative and interactive computations.

In [0]:
# Caching the DataFrame
df.cache()

# un-cache or unpersist data using the unpersist() method.
df.unpersist()

Out[124]: DataFrame[Name: string, City: string, City_lower: string]

#61. How to Divide a PySpark DataFrame randomly in a given ratio (0.8, 0.2)?

In [0]:
# Randomly split data (0.8, 0.2)

train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-3151542764839111> in <module> 
 1 # Randomly split data (0.8, 0.2) 
 2 
 ----> 3 train_data , test_data = data . randomSplit ( [ 0.8 , 0.2 ] , seed = 42 ) 

 AttributeError : 'list' object has no attribute 'randomSplit'

#62. How to build logistic regression in PySpark?

In [0]:
# Create a sample dataframe
data = spark.createDataFrame([
(0, 1.0, -1.0),
(1, 2.0, 1.0),
(1, 3.0, -2.0),
(0, 4.0, 1.0),
(1, 5.0, -3.0),
(0, 6.0, 2.0),
(1, 7.0, -1.0),
(0, 8.0, 3.0),
(1, 9.0, -2.0),
(0, 10.0, 2.0),
(1, 11.0, -3.0),
(0, 12.0, 1.0),
(1, 13.0, -1.0),
(0, 14.0, 2.0),
(1, 15.0, -2.0),
(0, 16.0, 3.0),
(1, 17.0, -3.0),
(0, 18.0, 1.0),
(1, 19.0, -1.0),
(0, 20.0, 2.0)
], ["label", "feat1", "feat2"])

In [0]:
### logistic regression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

# convert the feature columns into a single vector column using VectorAssembler
vecAssembler = VectorAssembler(inputCols=['feat1', 'feat2'], outputCol="features")
data = vecAssembler.transform(data)

# fit the logistic regression model
lr = LogisticRegression(featuresCol='features', labelCol='label')
lr_model = lr.fit(data)

# look at the coefficients and intercept of the logistic regression model
print(f"Coefficients: {str(lr_model.coefficients)}")
print(f"Intercept: {str(lr_model.intercept)}")

Coefficients: [0.020277736834151697,-1.6129606404964458]
Intercept: -0.22092825608663494

#63. How to convert the categorical string data into numerical data or index?

In [0]:
# Create a sample DataFrame
data = [('cat',), ('dog',), ('mouse',), ('fish',), ('dog',), ('cat',), ('mouse',)]
df = spark.createDataFrame(data, ["animal"])

df.show()

+------+
animal|
+------+
 cat|
 dog|
 mouse|
 fish|
 dog|
 cat|
 mouse|
+------+

In [0]:
### convert the categorical string data into numerical data or index
from pyspark.ml.feature import StringIndexer

# Initialize a StringIndexer
indexer = StringIndexer(inputCol='animal', outputCol='animalIndex')

# Fit the indexer to the DataFrame and transform the data
indexed = indexer.fit(df).transform(df)
indexed.display()

animal,animalIndex
cat,0.0
dog,1.0
mouse,2.0
fish,3.0
dog,1.0
cat,0.0
mouse,2.0


#64. How to calculate Correlation of two variables in a DataFrame?

In [0]:
# Create a sample dataframe
data = [Row(feature1=5, feature2=10, feature3=25),
Row(feature1=6, feature2=15, feature3=35),
Row(feature1=7, feature2=25, feature3=30),
Row(feature1=8, feature2=20, feature3=60),
Row(feature1=9, feature2=30, feature3=70)]
df = spark.createDataFrame(data)

df.show()

+--------+--------+--------+
feature1|feature2|feature3|
+--------+--------+--------+
 5| 10| 25|
 6| 15| 35|
 7| 25| 30|
 8| 20| 60|
 9| 30| 70|
+--------+--------+--------+

In [0]:
#### Calculate correlation
correlation = df.corr("feature1", "feature2")

print("Correlation between feature1 and feature2 :", correlation)

Correlation between feature1 and feature2 : 0.9000000000000001

#65. How to calculate Correlation Matrix?

In [0]:
# Create a sample dataframe
data = [Row(feature1=5, feature2=10, feature3=25),
Row(feature1=6, feature2=15, feature3=35),
Row(feature1=7, feature2=25, feature3=30),
Row(feature1=8, feature2=20, feature3=60),
Row(feature1=9, feature2=30, feature3=70)]
df = spark.createDataFrame(data)


df.show()

+--------+--------+--------+
feature1|feature2|feature3|
+--------+--------+--------+
 5| 10| 25|
 6| 15| 35|
 7| 25| 30|
 8| 20| 60|
 9| 30| 70|
+--------+--------+--------+

In [0]:
### Correlation Matrix
# Calculate Correlation Using Using MLlib
from pyspark.ml.stat import Correlation
from pyspark.sql.types import *
# Assemble feature vector
# Define the feature and label columns & Assemble the feature vector
vector_assembler = VectorAssembler(inputCols=["feature1", "feature2", "feature3"], outputCol="features")
data_vector = vector_assembler.transform(df).select("features")

# Calculate correlation
correlation_matrix = Correlation.corr(data_vector, "features").head()[0]

print(correlation_matrix)

correlation_list = correlation_matrix.toArray().tolist()
# Define the schema for the DataFrame
schema = StructType([StructField(f"col{i}", FloatType(), True) for i in range(len(correlation_list))])
# Create the DataFrame
correlation_df = spark.createDataFrame(correlation_list, schema)

correlation_df.display()

DenseMatrix([[1. , 0.9 , 0.91779992],
 [0.9 , 1. , 0.67837385],
 [0.91779992, 0.67837385, 1. ]])

col0,col1,col2
1.0,0.9,0.9177999
0.9,1.0,0.6783739
0.9177999,0.6783739,1.0


#66. How to calculate VIF (Variance Inflation Factor ) for set of variables in a DataFrame?

In [0]:
# Create a sample dataframe
data = [Row(feature1=5, feature2=10, feature3=25),
Row(feature1=6, feature2=15, feature3=35),
Row(feature1=7, feature2=25, feature3=30),
Row(feature1=8, feature2=20, feature3=60),
Row(feature1=9, feature2=30, feature3=70)]
df = spark.createDataFrame(data)

df.show()

+--------+--------+--------+
feature1|feature2|feature3|
+--------+--------+--------+
 5| 10| 25|
 6| 15| 35|
 7| 25| 30|
 8| 20| 60|
 9| 30| 70|
+--------+--------+--------+

In [0]:
from pyspark.sql import SparkSession, Row
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

def calculate_vif(data, features):
    vif_dict = {}

    for feature in features:
        non_feature_cols = [col for col in features if col != feature]
        assembler = VectorAssembler(inputCols=non_feature_cols, outputCol="features")
        lr = LinearRegression(featuresCol='features', labelCol=feature)

        model = lr.fit(assembler.transform(data))
        vif = 1 / (1 - model.summary.r2)

        vif_dict[feature] = vif

    return vif_dict

features = ['feature1', 'feature2', 'feature3']
vif_values = calculate_vif(df, features)

for feature, vif in vif_values.items():
    print(f'VIF for {feature}: {vif}')

VIF for feature1: 66.2109374999998
VIF for feature2: 19.335937499999957
VIF for feature3: 23.30468749999998

#67. How to perform Chi-Square test?

In [0]:
# Create a sample dataframe
data = [(1, 0, 0, 1, 1),
(2, 0, 1, 0, 0),
(3, 1, 0, 0, 0),
(4, 0, 0, 1, 1),
(5, 0, 1, 1, 0)]

df = spark.createDataFrame(data, ["id", "feature1", "feature2", "feature3", "label"])

df.show()

+---+--------+--------+--------+-----+
 id|feature1|feature2|feature3|label|
+---+--------+--------+--------+-----+
 1| 0| 0| 1| 1|
 2| 0| 1| 0| 0|
 3| 1| 0| 0| 0|
 4| 0| 0| 1| 1|
 5| 0| 1| 1| 0|
+---+--------+--------+--------+-----+

In [0]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["feature1", "feature2", "feature3"], outputCol="features")
df = assembler.transform(df)

from pyspark.ml.stat import ChiSquareTest

r = ChiSquareTest.test(df, "features", "label").head()
print("pValues: " + str(r.pValues))
print("degreesOfFreedom: " + str(r.degreesOfFreedom))
print("statistics: " + str(r.statistics))

pValues: [0.36131042852617856,0.13603712811414348,0.1360371281141436]
degreesOfFreedom: [1, 1, 1]
statistics: [0.8333333333333335,2.2222222222222228,2.2222222222222223]

#68. How to calculate the Standard Deviation?

In [0]:
# Sample data
data = [("James", "Sales", 3000),
("Michael", "Sales", 4600),
("Robert", "Sales", 4100),
("Maria", "Finance", 3000),
("James", "Sales", 3000),
("Scott", "Finance", 3300),
("Jen", "Finance", 3900),
("Jeff", "Marketing", 3000),
("Kumar", "Marketing", 2000),
("Saif", "Sales", 4100)]

# Create DataFrame
df = spark.createDataFrame(data, ["Employee", "Department", "Salary"])

df.show()

+--------+----------+------+
Employee|Department|Salary|
+--------+----------+------+
 James| Sales| 3000|
 Michael| Sales| 4600|
 Robert| Sales| 4100|
 Maria| Finance| 3000|
 James| Sales| 3000|
 Scott| Finance| 3300|
 Jen| Finance| 3900|
 Jeff| Marketing| 3000|
 Kumar| Marketing| 2000|
 Saif| Sales| 4100|
+--------+----------+------+

In [0]:
### Standard Deviation
from pyspark.sql.functions import stddev

df_result = df.select(stddev("Salary").alias("stddev"))

df_result.display()

stddev
765.9416862050705


#69. How to calculate missing value percentage in each column?

In [0]:
# Create a sample dataframe
data = [("John", "Doe", None),
(None, "Smith", "New York"),
("Mike", "Smith", None),
("Anna", "Smith", "Boston"),
(None, None, None)]

df = spark.createDataFrame(data, ["FirstName", "LastName", "City"])

df.show()

+---------+--------+--------+
FirstName|LastName| City|
+---------+--------+--------+
 John| Doe| null|
 null| Smith|New York|
 Mike| Smith| null|
 Anna| Smith| Boston|
 null| null| null|
+---------+--------+--------+

In [0]:
# Calculate the total number of rows in the dataframe
total_rows = df.count()

# For each column calculate the number of null values and then calculate the percentage
for column in df.columns:
    null_values = df.filter(df[column].isNull()).count()
    missing_percentage = (null_values / total_rows) * 100
    print(f"Missing values in {column}: {missing_percentage}%")


Missing values in FirstName: 40.0%
Missing values in LastName: 20.0%
Missing values in City: 60.0%

#70. How to get the names of DataFrame objects that have been created in an environment?

In [0]:
## DataFrame objects that have been created
from pyspark.sql import dataframe
from pyspark.sql.dataframe import DataFrame

dataframe_names = [name for name, obj in globals().items() if isinstance(obj, DataFrame)]

for name in dataframe_names:
    print(name)

_
df
df_index
df_min
df_freq
df_tmp
df_result
df_renamed
df_binned
df_A
df_B
df_src
df_with_diag_zero
indexed_df
encoded_df
df_imputed
df_grouped
mode_df
pysparkDF
_124
indexed
data_vector
correlation_df

#71. Reading and Writing Files

##1. CSV Files

In [0]:
###To read a CSV file using PySpark, you can use the read.csv() method:

#infer schema
csv_file = "path/to/your/csv/file.csv"
df_csv = spark.read.csv(csv_file, header=True, inferSchema=True)

from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType

customschema = StructType([
    StructField("A", StringType(), True),
    StructField("B", DoubleType(), True),
    StructField("C", TimestampType(), True)
])
#header="true": Indicates that the first row contains column names.
#multiline="true": Allows reading multiline records.
#schema=customschema: Specifies the custom schema you defined.
#load(destinationPath): Replace destinationPath with the path to your CSV file.

df_1 = spark.read.format("csv") \
    .options(header="true", multiline="true") \
    .schema(customschema) \
    .load(destinationPath)

--------------------------------------------------------------------------- 
 IllegalArgumentException Traceback (most recent call last)
 <command-1177762865691178> in <module> 
 3 #infer schema 
 4 csv_file = "path/to/your/csv/file.csv" 
 ----> 5 df_csv = spark . read . csv ( csv_file , header = True , inferSchema = True ) 
 6 
 7 from pyspark . sql . types import StructType , StructField , StringType , DoubleType , TimestampType

 /databricks/spark/python/pyspark/sql/readwriter.py in csv (self, path, schema, sep, encoding, quote, escape, comment, header, inferSchema, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, nullValue, nanValue, positiveInf, negativeInf, dateFormat, timestampFormat, maxColumns, maxCharsPerColumn, maxMalformedLogPerPartition, mode, columnNameOfCorruptRecord, multiLine, charToEscapeQuoteEscaping, samplingRatio, enforceSchema, emptyValue, locale, lineSep, pathGlobFilter, recursiveFileLookup, modifiedBefore, modifiedAfter, unescapedQuoteHandling) 
 762 path = [ path ] 
 763 if type ( path ) == list : 
 --> 764 return self . _df ( self . _jreader . csv ( self . _spark . _sc . _jvm . PythonUtils . toSeq ( path ) ) ) 
 765 elif isinstance ( path , RDD ) : 
 766 def func ( iterator ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 IllegalArgumentException : Path must be absolute: path/to/your/csv/file.csv

In [0]:
#Now that you have your data in a DataFrame, you can write it back to a CSV file using the write.csv() method:
output_path = "path/to/output/csv/file.csv"
df_csv.write.csv(output_path, header=True, mode="overwrite")

df.write.option("header", "false").option("sep", "|").csv("/path/to/save/output.csv")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1177762865691179> in <module> 
 1 #Now that you have your data in a DataFrame, you can write it back to a CSV file using the write.csv() method: 
 2 output_path = "path/to/output/csv/file.csv" 
 ----> 3 df_csv . write . csv ( output_path , header = True , mode = "overwrite" ) 
 4 
 5 df . write . option ( "header" , "false" ) . option ( "sep" , "|" ) . csv ( "/path/to/save/output.csv" ) 

 NameError : name 'df_csv' is not defined

##2. Parquet files

In [0]:
###To read a Parquet file using PySpark, you can use the read.parquet() method:
parquet_file = "path/to/your/parquet/file.parquet"
df_parquet = spark.read.parquet(parquet_file)

#To write the data back to a Parquet file, use the write.parquet() method:
output_path = "path/to/output/parquet/file.parquet"
df_parquet.write.parquet(output_path, mode="overwrite")

--------------------------------------------------------------------------- 
 IllegalArgumentException Traceback (most recent call last)
 <command-1177762865691182> in <module> 
 1 ###To read a Parquet file using PySpark, you can use the read.parquet() method: 
 2 parquet_file = "path/to/your/parquet/file.parquet" 
 ----> 3 df_parquet = spark . read . parquet ( parquet_file ) 
 4 
 5 #To write the data back to a Parquet file, use the write.parquet() method: 

 /databricks/spark/python/pyspark/sql/readwriter.py in parquet (self, *paths, **options) 
 483 int96RebaseMode=int96RebaseMode)
 484 
 --> 485 return self . _df ( self . _jreader . parquet ( _to_seq ( self . _spark . _sc , paths ) ) ) 
 486 
 487 def text(self, paths, wholetext=False, lineSep=None, pathGlobFilter=None,

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 IllegalArgumentException : Path must be absolute: path/to/your/parquet/file.parquet

#72. Temporary SQL table

In [0]:
### Temporary SQL table
#We’ll create a sample DataFrame using a list of dictionaries and register the 
# DataFrame as a temporary SQL table to perform SQL operations

data = [
    {"name": "Alice", "age": 30, "city": "New York"},
    {"name": "Bob", "age": 25, "city": "San Francisco"},
    {"name": "Charlie", "age": 35, "city": "Los Angeles"}
]

df = spark.createDataFrame(data)

df.createOrReplaceTempView("people")

query = "SELECT * FROM people WHERE age >= 30"
result_df = spark.sql(query)
result_df.display()

age,city,name
30,New York,Alice
35,Los Angeles,Charlie


#73. Dropping Columns Using Regex Pattern

In [0]:
data = [("Alice", 30, "New York", "F"),
        ("Bob", 28, "San Francisco", "M"),
        ("Cathy", 29, "Los Angeles", "F"),
        ("David", 32, "Chicago", "M")]

columns = ["name", "age", "city", "gender"]

df = spark.createDataFrame(data, columns)
df.show()

+-----+---+-------------+------+
 name|age| city|gender|
+-----+---+-------------+------+
Alice| 30| New York| F|
 Bob| 28|San Francisco| M|
Cathy| 29| Los Angeles| F|
David| 32| Chicago| M|
+-----+---+-------------+------+

In [0]:
from pyspark.sql.functions import col
import re

regex_pattern = "gender|age"
df = df.select([col(c) for c in df.columns if not re.match(regex_pattern, c)])

df.show()

+-----+-------------+
 name| city|
+-----+-------------+
Alice| New York|
 Bob|San Francisco|
Cathy| Los Angeles|
David| Chicago|
+-----+-------------+

#74. Joins

In [0]:
### Inner Join:
# Combines rows from both DataFrames where the join condition is met.
# Only the matching rows are included in the result.

joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "inner")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4490529496103679> in <module> 
 3 # Only the matching rows are included in the result. 
 4 
 ----> 5 joined_df = empDF . join ( deptDF , empDF [ "emp_dept_id" ] == deptDF [ "dept_id" ] , "inner" ) 

 NameError : name 'empDF' is not defined

In [0]:
### Inner Join:
# Combines rows from both DataFrames where the join condition is met.
# Only the matching rows are included in the result.

joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "inner")
### Full Outer Join:
# Includes all rows from both DataFrames.
# If there’s no match, the missing values are filled with null.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "outer")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4490529496103680> in <module> 
 3 # Only the matching rows are included in the result. 
 4 
 ----> 5 joined_df = empDF . join ( deptDF , empDF [ "emp_dept_id" ] == deptDF [ "dept_id" ] , "inner" ) 
 6 ### Full Outer Join: 
 7 # Includes all rows from both DataFrames. 

 NameError : name 'empDF' is not defined

In [0]:
### Inner Join:
# Combines rows from both DataFrames where the join condition is met.
# Only the matching rows are included in the result.

joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "inner")
### Full Outer Join:
# Includes all rows from both DataFrames.
# If there’s no match, the missing values are filled with null.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "outer")
### Left Outer Join:
# Includes all rows from the left DataFrame and matching rows from the right DataFrame.
# Non-matching rows from the right DataFrame are filled with null.

joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "left")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4490529496103681> in <module> 
 3 # Only the matching rows are included in the result. 
 4 
 ----> 5 joined_df = empDF . join ( deptDF , empDF [ "emp_dept_id" ] == deptDF [ "dept_id" ] , "inner" ) 
 6 ### Full Outer Join: 
 7 # Includes all rows from both DataFrames. 

 NameError : name 'empDF' is not defined

In [0]:
### Inner Join:
# Combines rows from both DataFrames where the join condition is met.
# Only the matching rows are included in the result.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "inner")

### Full Outer Join:
# Includes all rows from both DataFrames.
# If there’s no match, the missing values are filled with null.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "outer")

### Left Outer Join:
# Includes all rows from the left DataFrame and matching rows from the right DataFrame.
# Non-matching rows from the right DataFrame are filled with null.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "left")

### Right Outer Join:
# Similar to left outer join but includes all rows from the right DataFrame.
# Non-matching rows from the left DataFrame are filled with null.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "right")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4490529496103682> in <module> 
 2 # Combines rows from both DataFrames where the join condition is met. 
 3 # Only the matching rows are included in the result. 
 ----> 4 joined_df = empDF . join ( deptDF , empDF [ "emp_dept_id" ] == deptDF [ "dept_id" ] , "inner" ) 
 5 
 6 ### Full Outer Join: 

 NameError : name 'empDF' is not defined

In [0]:
### Inner Join:
# Combines rows from both DataFrames where the join condition is met.
# Only the matching rows are included in the result.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "inner")

### Full Outer Join:
# Includes all rows from both DataFrames.
# If there’s no match, the missing values are filled with null.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "outer")

### Left Outer Join:
# Includes all rows from the left DataFrame and matching rows from the right DataFrame.
# Non-matching rows from the right DataFrame are filled with null.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "left")

### Right Outer Join:
# Similar to left outer join but includes all rows from the right DataFrame.
# Non-matching rows from the left DataFrame are filled with null.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "right")

### Left Anti Join:
# Returns rows from the left DataFrame that do not have a match in the right DataFrame.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "left_anti")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4490529496103683> in <module> 
 2 # Combines rows from both DataFrames where the join condition is met. 
 3 # Only the matching rows are included in the result. 
 ----> 4 joined_df = empDF . join ( deptDF , empDF [ "emp_dept_id" ] == deptDF [ "dept_id" ] , "inner" ) 
 5 
 6 ### Full Outer Join: 

 NameError : name 'empDF' is not defined

In [0]:
### Inner Join:
# Combines rows from both DataFrames where the join condition is met.
# Only the matching rows are included in the result.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "inner")

### Full Outer Join:
# Includes all rows from both DataFrames.
# If there’s no match, the missing values are filled with null.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "outer")

### Left Outer Join:
# Includes all rows from the left DataFrame and matching rows from the right DataFrame.
# Non-matching rows from the right DataFrame are filled with null.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "left")

### Right Outer Join:
# Similar to left outer join but includes all rows from the right DataFrame.
# Non-matching rows from the left DataFrame are filled with null.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "right")

### Left Anti Join:
# Returns rows from the left DataFrame that do not have a match in the right DataFrame.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "left_anti")

### Left Semi Join:
# Returns rows from the left DataFrame that have at least one match in the right DataFrame.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "left_semi")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4490529496103684> in <module> 
 2 # Combines rows from both DataFrames where the join condition is met. 
 3 # Only the matching rows are included in the result. 
 ----> 4 joined_df = empDF . join ( deptDF , empDF [ "emp_dept_id" ] == deptDF [ "dept_id" ] , "inner" ) 
 5 
 6 ### Full Outer Join: 

 NameError : name 'empDF' is not defined

In [0]:
### Inner Join:
# Combines rows from both DataFrames where the join condition is met.
# Only the matching rows are included in the result.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "inner")

### Full Outer Join:
# Includes all rows from both DataFrames.
# If there’s no match, the missing values are filled with null.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "outer")

### Left Outer Join:
# Includes all rows from the left DataFrame and matching rows from the right DataFrame.
# Non-matching rows from the right DataFrame are filled with null.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "left")

### Right Outer Join:
# Similar to left outer join but includes all rows from the right DataFrame.
# Non-matching rows from the left DataFrame are filled with null.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "right")

### Left Anti Join:
# Returns rows from the left DataFrame that do not have a match in the right DataFrame.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "left_anti")

### Left Semi Join:
# Returns rows from the left DataFrame that have at least one match in the right DataFrame.
joined_df = empDF.join(deptDF, empDF["emp_dept_id"] == deptDF["dept_id"], "left_semi")

### Cross Join:
# Generates all possible combinations of rows from both DataFrames.
# Use with caution as it can result in a large output.
joined_df = empDF.crossJoin(deptDF.select("dept_id"))

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4490529496103685> in <module> 
 2 # Combines rows from both DataFrames where the join condition is met. 
 3 # Only the matching rows are included in the result. 
 ----> 4 joined_df = empDF . join ( deptDF , empDF [ "emp_dept_id" ] == deptDF [ "dept_id" ] , "inner" ) 
 5 
 6 ### Full Outer Join: 

 NameError : name 'empDF' is not defined

#75. Remove extra spaces from the specified column

In [0]:
def remove_extra_spaces(df, column_name):
    # Remove extra spaces from the specified column
    df_transformed = df.withColumn(column_name, regexp_replace(col(column_name), "\\s+", " "))
    return df_transformed

transformed_df = remove_extra_spaces(df, "name")
transformed_df.show()

+-----+-------------+
 name| city|
+-----+-------------+
Alice| New York|
 Bob|San Francisco|
Cathy| Los Angeles|
David| Chicago|
+-----+-------------+

#76. StringIndexer

In [0]:
# The StringIndexer is a vital PySpark feature that helps convert categorical string columns in a DataFrame into numerical indices.

data = [("A", 10),("A", 20),("B", 30),("B", 20),("B", 30),("C", 40),("C", 10),("D", 10)]
columns = ["Categories", "Value"]
df = spark.createDataFrame(data, columns)
df.show()

+----------+-----+
Categories|Value|
+----------+-----+
 A| 10|
 A| 20|
 B| 30|
 B| 20|
 B| 30|
 C| 40|
 C| 10|
 D| 10|
+----------+-----+

In [0]:
### StringIndexer
# The StringIndexer is a vital PySpark feature that helps convert 
# categorical string columns in a DataFrame into numerical indices.

from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer

# StringIndexer Initialization
indexer = StringIndexer(inputCol="Categories", outputCol="Categories_Indexed")
indexerModel = indexer.fit(df)

# Transform the DataFrame using the fitted StringIndexer model
indexed_df = indexerModel.transform(df)
indexed_df.show()

+----------+-----+------------------+
Categories|Value|Categories_Indexed|
+----------+-----+------------------+
 A| 10| 1.0|
 A| 20| 1.0|
 B| 30| 0.0|
 B| 20| 0.0|
 B| 30| 0.0|
 C| 40| 2.0|
 C| 10| 2.0|
 D| 10| 3.0|
+----------+-----+------------------+

In [0]:
#Handling unseen labels in test data
# In real-world scenarios, your model may encounter unseen labels in the test data. By default, StringIndexer throws an
# error when it comes across an unseen label. To handle such cases, you can set the handleInvalid1 parameter to 'skip',
# 'keep', or 'error', depending on your requirements.
 
data = [("A", 10),("A", 20),("B", 30),("B", 20),("B", 30),("C", 40),("C", 10),("D", 10)]
columns = ["Categories", "Value"]
train_df = spark.createDataFrame(data, columns)
train_df.show()

+----------+-----+
Categories|Value|
+----------+-----+
 A| 10|
 A| 20|
 B| 30|
 B| 20|
 B| 30|
 C| 40|
 C| 10|
 D| 10|
+----------+-----+

In [0]:
#Initialize the StringIndexer uaing handleInvalid="keep" and fit on train_df where dataframe train_df is having four categories A, B, C, D

indexer = StringIndexer(inputCol="Categories", outputCol="Categories_Indexed", handleInvalid="keep")
train_indexerModel = indexer.fit(train_df)

#Create Test DataFrame
data = [("A", 15),("A", 22),("B", 38),("B", 20),("C", 18),("E", 19),("F", 17)]
columns = ["Categories", "Value"]
test_df = spark.createDataFrame(data, columns)
test_df.show()

#ransform DataFrame test_df where in dataframe test_df is having tow new categories E, F and category D is missing
test_indexed_df = train_indexerModel.transform(test_df)
test_indexed_df.show()


+----------+-----+
Categories|Value|
+----------+-----+
 A| 15|
 A| 22|
 B| 38|
 B| 20|
 C| 18|
 E| 19|
 F| 17|
+----------+-----+

+----------+-----+------------------+
Categories|Value|Categories_Indexed|
+----------+-----+------------------+
 A| 15| 1.0|
 A| 22| 1.0|
 B| 38| 0.0|
 B| 20| 0.0|
 C| 18| 2.0|
 E| 19| 4.0|
 F| 17| 4.0|
+----------+-----+------------------+

#77. Reversing StringIndexer transformation with IndexToString

In [0]:
#In some cases, you may need to reverse the transformation applied by StringIndexer to interpret your model’s predictions.

# Example Data
data = [("A", 10),("A", 20),("B", 30),("B", 20),("B", 30),("C", 40),("C", 10),("D", 10)]
columns = ["Categories", "Value"]

df = spark.createDataFrame(data, columns)
df.show()

+----------+-----+
Categories|Value|
+----------+-----+
 A| 10|
 A| 20|
 B| 30|
 B| 20|
 B| 30|
 C| 40|
 C| 10|
 D| 10|
+----------+-----+

In [0]:
### Reverse StringIndexer
# Initialize the StringIndexer and Transform the 
# DataFrame using the fitted StringIndexer model
# StringIndexer Initialization
indexer = StringIndexer(inputCol="Categories", outputCol="Categories_Indexed")
indexerModel = indexer.fit(df)

# Transform the DataFrame using the fitted StringIndexer model
indexed_df = indexerModel.transform(df)
indexed_df.show()

#Import the IndexToString transformer
from pyspark.ml.feature import IndexToString

#Initialize the IndexToString
index_to_string = IndexToString(inputCol="Categories_Indexed", outputCol="Pred_Category",
                                labels=indexerModel.labels)

# Transform the DataFrame
result_df = index_to_string.transform(indexed_df)

result_df.show()

+----------+-----+------------------+
Categories|Value|Categories_Indexed|
+----------+-----+------------------+
 A| 10| 1.0|
 A| 20| 1.0|
 B| 30| 0.0|
 B| 20| 0.0|
 B| 30| 0.0|
 C| 40| 2.0|
 C| 10| 2.0|
 D| 10| 3.0|
+----------+-----+------------------+

+----------+-----+------------------+-------------+
Categories|Value|Categories_Indexed|Pred_Category|
+----------+-----+------------------+-------------+
 A| 10| 1.0| A|
 A| 20| 1.0| A|
 B| 30| 0.0| B|
 B| 20| 0.0| B|
 B| 30| 0.0| B|
 C| 40| 2.0| C|
 C| 10| 2.0| C|
 D| 10| 3.0| D|
+----------+-----+------------------+-------------+

In [0]:
# Read static DataFrame
staticDf = spark.read.format("csv").load("static_data.csv")

# Read streaming DataFrame
streamingDf = spark.readStream.format("csv").load("streaming_data.csv")

# Perform an inner equi-join with a static DataFrame
joinedDf = streamingDf.join(staticDf, "type")


In [0]:
### Streaming Data
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.appName("CSVAppendExample").getOrCreate()

# Define your schema
schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("city", StringType(), True)
])

newCsvPath = "path/to/new_data.csv"
newCsvDf = spark.read.csv(newCsvPath, header=True, schema=schema)

appendedDf = existingDf.union(newCsvDf)

appendedDf.write.csv("path/to/output.csv", mode="append", header=True)
